# Mount drive

In [165]:
#@title Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Setup

In [168]:
#@title Installations
!pip install --upgrade descartes geopandas plotly pyshp rasterio rtree scikit-misc shapely &> /dev/null



In [169]:
#@title Imports TODO
%load_ext autoreload
%autoreload 2

ROOT = './'

import sys
sys.path.append(ROOT + 'src/py/')

import math
import random
import time

import geopandas as gpd
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from datetime import datetime
from scipy.stats import pearsonr, spearmanr, ttest_1samp, ttest_ind
from shapely.geometry import Point, Polygon
from sklearn.metrics import auc
from sklearn.neighbors import NearestNeighbors
from skmisc import loess

import constants as c
from constants import PALETTE as pal
from bing_tile import bing_tile_to_polygon
from cv_utils import get_assignments_df
from misc_utils import standardize, tile_weighted_avg
from pca_utils import fill_missing, run_pca
from plot_utils import get_mapping_df, scatterplot, __plot_loess_helper

pd.set_option('display.max_columns', 100)

DATA_PATH = ROOT + 'data/'
NOT_INC_DATA_PATH = 'poverty_map_paper/data/'
GEO_PATH = DATA_PATH + 'geo/'
EST_PATH = DATA_PATH + 'generated_estimates/'

geodata_path = GEO_PATH + 'updated_wards/NGA_Ward_Map.shp'
nlss_data_path = DATA_PATH + 'nigeria_hh_survey.csv'

today_str = str(datetime.today().month).zfill(2) + \
            str(datetime.today().year % 1000)
FIG_PATH = ROOT + 'output/figures/%s/' % today_str
OUT_DIR = ROOT + 'output/figure_data/%s/' % today_str

! mkdir -p $FIG_PATH && mkdir -p $OUT_DIR

import warnings
import matplotlib.cbook
warnings.filterwarnings('ignore', category=matplotlib.cbook.mplDeprecation)

!mkdir -p $FIG_PATH

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read in and process data

In [170]:
#@title Read in administrative boundaries
country = gpd.read_file(DATA_PATH + 'admin_boundaries/country_boundary.shp')
states = gpd.read_file(DATA_PATH + 'admin_boundaries/states.shp')
lgas = gpd.read_file(DATA_PATH + 'admin_boundaries/lgas.shp')
wards = gpd.read_file(DATA_PATH + 'admin_boundaries/wards.shp')\
           .rename({'ward_origi': 'ward_original_name'}, axis=1)

In [171]:
#@title Read in NLSS

def get_survey_geom(df):
    return Point(df['hh_gps_longitude'], df['hh_gps_latitude'])

nlss_data_path = NOT_INC_DATA_PATH + 'nlss/nigeria_hh_survey.csv'
nlss_data = pd.read_csv(nlss_data_path)\
                .dropna(subset=['hh_gps_longitude', 'hh_gps_latitude'])\
                .drop(['state', 'lga'], axis=1)
nlss_data['geometry'] = nlss_data.apply(get_survey_geom, axis=1)
nlss_data['weight'] = nlss_data['popw'] # Use population weights, not household
nlss_data = gpd.GeoDataFrame(nlss_data, crs=c.DEFAULT_CRS)
nlss_data = gpd.sjoin(nlss_data, wards[['ward', 'lga', 'state', 'geometry']],
                        how='left', op='intersects')\
                 .drop(['index_right'], axis=1)

nlss_data['log_con'] = nlss_data['totcons_adj'].apply(lambda x: np.log(x))


/usr/local/lib/python3.7/dist-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if self.run_code(code, result):


In [173]:
#@title Read in satellite poverty estimates
didl_orig_state = tile_weighted_avg(states, didl_orig_all, 'state', 'estimated_rwi')\
    .rename({'estimated_rwi': 'didl_orig_rwi', 'weighted_pop': 'pop'}, axis=1)
didl_orig_lga = tile_weighted_avg(lgas, didl_orig_all, 'lga', 'estimated_rwi')\
    .rename({'estimated_rwi': 'didl_orig_rwi', 'weighted_pop': 'pop'}, axis=1)
didl_orig_ward = tile_weighted_avg(wards, didl_orig_all, 'ward', 'estimated_rwi')\
    .rename({'estimated_rwi': 'didl_orig_rwi', 'weighted_pop': 'pop'}, axis=1)

didl_orig_all = pd.read_csv(NOT_INC_DATA_PATH + 'estimates/NG_RWI.csv')
pop_data = pd.read_csv(NOT_INC_DATA_PATH + 'ng_pop.csv') ##
didl_orig_all['geometry'] = didl_orig_all.apply(lambda x: bing_tile_to_polygon(x), axis=1)
didl_orig_all = gpd.GeoDataFrame(didl_orig_all.drop('iso2', axis=1)\
                                              .rename({'rwi': 'estimated_rwi'}, axis=1)
                                              .merge(pop_data, on=['x', 'y']),
                                 crs=c.DEFAULT_CRS)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [174]:
#@title Read in DHS -- change for urban only

rwi_var = 'hv271'

dhs = pd.read_stata(NOT_INC_DATA_PATH + 'dhs/stata_hh/NGHR7AFL.DTA')
locs = gpd.read_file(NOT_INC_DATA_PATH + 'dhs/gps_data/NGGE7AFL.shp')

# dhs = dhs[dhs['hv025'] == 'urban']

# Use DHS-generated RWI
dhs_eval = dhs[['hhid', 'hv001', 'shstate', 'hv005', rwi_var]]\
              .rename({'hv001': 'cluster_id', 'hv005': 'weight', rwi_var: 'dhs_rwi'},
                      axis=1)
dhs_eval['weight'] /= 1000000

rename_states = {'fct abuja': 'federal capital territory',
                 'nasarawa': 'nassarawa'}
dhs_eval['shstate'] = dhs_eval['shstate'].replace(rename_states)

dhs_eval = locs[['DHSCLUST', 'geometry']]\
           .merge(dhs_eval, left_on='DHSCLUST', right_on='cluster_id', how='right')\
           .drop('DHSCLUST', axis=1)
dhs_eval = gpd.sjoin(dhs_eval,
                     wards.drop_duplicates(subset='ward')[['ward', 'lga', 'state', 'geometry']],
                     how='left', op='within')\
              .rename({'state': 'geo_state'}, axis=1)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if self.run_code(code, result):


In [175]:
#@title Add population to administrative boundary dfs
states = states.merge(didl_orig_state[['state', 'pop']], how='left')
lgas = lgas.merge(didl_orig_lga[['lga', 'pop']], how='left')
wards = wards.merge(didl_orig_ward.drop_duplicates(subset='ward')[['ward', 'pop']], how='left')


In [176]:
#@title Generate DHS-like rwi for NLSS data -- change for urban only

weights_path = DATA_PATH + 'dhs/rwi_weights/rwi_mapping_combined.csv'
rwi_weights = pd.read_csv(weights_path)
rwi_weights = rwi_weights[['nlss_name', 'one_weight', 'zero_weight']].dropna()
rwi_weights = rwi_weights.groupby('nlss_name').mean().reset_index()

factor_cols = ['main_water',
              'sanitation_type',
              'cookstovetype',
              'rooftype',
              'wallstype',
              'floortype']  
cont_cols = ['numasset_radio',
             'numasset_tv',
             'numasset_computer',
             'numasset_fridge',
             'numasset_table',
             'numasset_chair',
             'numasset_bed',
             'numasset_sofa',
             'numasset_ac',
             'numasset_iron',
             'numasset_generator',
             'numasset_fan',
             'has_mobile_phone',
             'numasset_bike',
             'numasset_motorbike',
             'numasset_car',
             'electricity']

nlss_pca_df = fill_missing(nlss_data.copy(), factor_cols, continuous_cols=None)
nlss_pca_df['electricity'] = nlss_pca_df['electricity'].apply(lambda x: 1 if x == 'yes' else 0)
nlss_pca_df['has_mobile_phone'] = nlss_pca_df['numasset_smartphone'] + \
                                  nlss_pca_df['numasset_regmobilephone']

for col in cont_cols:
    nlss_pca_df[col] = nlss_pca_df[col].apply(lambda x: 1 if x > 0 else 0)

nlss_pca_df[cont_cols] = nlss_pca_df[cont_cols].fillna(nlss_pca_df[cont_cols].median())

dummy_colnames = []
for col in factor_cols:
    dummies = pd.get_dummies(nlss_pca_df[col], prefix=col)
    dummy_colnames += list(dummies.columns)
    nlss_pca_df = pd.concat([nlss_pca_df, dummies], axis=1)

nlss_pca_df['memsleep'] = nlss_pca_df['hhsize'] / nlss_pca_df['numsleepingrooms']
nlss_pca_df = nlss_pca_df[['hhid', 'memsleep'] + dummy_colnames + cont_cols]

def get_catchall_dict(factor_cols, all_cols):
    catchall_dict = {}
    for col in factor_cols:
        for catchall_col in all_cols:
            if catchall_col.startswith(col) and 'other' in catchall_col.lower():
                catchall_dict[col] = catchall_col
                break
    assert len(catchall_dict) == len(factor_cols)
    return catchall_dict

def get_catchall_column(col_to_match, factor_cols, catchall_dict):
    for fc in factor_cols:
        if col_to_match.startswith(fc):
            return catchall_dict[fc]
    raise Exception('no catchall column for variable')

catchall_dict = get_catchall_dict(factor_cols, nlss_pca_df.columns)
for col in nlss_pca_df.columns:
    # DHS has no category - combine with "other" for appropriate group
    if col not in set(rwi_weights['nlss_name']) and col in set(dummy_colnames):
        catchall_col = get_catchall_column(col, factor_cols, catchall_dict)
        nlss_pca_df[catchall_col] += nlss_pca_df[col]

nlss_pca_df = nlss_pca_df[list(rwi_weights['nlss_name']) + ['hhid']]
for col in nlss_pca_df[list(rwi_weights['nlss_name'])]:
    vals = rwi_weights.loc[rwi_weights['nlss_name'] == col].iloc[0]
    nlss_pca_df[col] = nlss_pca_df[col].apply(lambda x: vals['zero_weight'] if x == 0 else
                                                        vals['one_weight'])

nlss_pca_df['dhs_like_rwi'] = standardize(
    nlss_pca_df[list(rwi_weights['nlss_name'])].sum(axis=1))
nlss_rwi_dhs_like = nlss_pca_df[['dhs_like_rwi', 'hhid']]


In [177]:
#@title Aggregate NLSS rwi over administrative units

nlss_rwi_df = nlss_rwi_dhs_like
nlss_rwi_col = 'dhs_like_rwi'

# nlss_rwi_df = nlss_rwi_pca
# nlss_rwi_col = 'survey_rwi_full'

nlss_hhs = nlss_rwi_df.merge(nlss_data[['hhid', 'popw', 'ward', 'lga', 'state']])\
                      .rename({nlss_rwi_col: 'nlss_rwi'}, axis=1)
nlss_hhs['nlss_rwi_w'] = nlss_hhs['nlss_rwi'] * nlss_hhs['popw']

nlss_state = nlss_hhs.groupby('state')[['popw', 'nlss_rwi_w']].sum().reset_index()
nlss_state['nlss_rwi'] = standardize(nlss_state['nlss_rwi_w'] / nlss_state['popw'])
nlss_state = nlss_state[['state', 'nlss_rwi', 'popw']]

nlss_lga = nlss_hhs.groupby('lga')[['nlss_rwi']].mean().reset_index()
nlss_lga['nlss_rwi'] = standardize(nlss_lga['nlss_rwi'])

nlss_ward = nlss_hhs.groupby('ward')[['nlss_rwi']].mean().reset_index()
nlss_ward['nlss_rwi'] = standardize(nlss_ward['nlss_rwi'])


In [178]:
#@title Aggregate DHS rwi over administrative units
dhs_eval['dhs_rwi_w'] = dhs_eval['dhs_rwi'] * dhs_eval['weight']

dhs_state = dhs_eval.groupby('geo_state')[['dhs_rwi_w', 'weight']].sum().reset_index()\
                    .rename({'geo_state': 'state'}, axis=1)
dhs_state['dhs_rwi'] = standardize(dhs_state['dhs_rwi_w'] / dhs_state['weight'])
dhs_state = dhs_state[['state', 'dhs_rwi']]

dhs_lga = dhs_eval.groupby('lga')[['dhs_rwi']].mean().reset_index()
dhs_lga['dhs_rwi'] = standardize(dhs_lga['dhs_rwi'])

dhs_ward = dhs_eval.groupby('ward')[['dhs_rwi']].mean().reset_index()
dhs_ward['dhs_rwi'] = standardize(dhs_ward['dhs_rwi'])


# Shared data & helpers

In [179]:
#@title Make poverty line df

pl = nlss_data['pl'].iloc[0]
ultra_pl = pl / 2

def get_weighted_quantile(df, q_col, weight_col, out_col):
    df = df.sort_values(q_col)
    df['cum_weight'] = df[weight_col].cumsum()
    total = df[weight_col].sum()
    dedup = df.drop_duplicates(q_col, keep='last')
    dedup[out_col] = dedup['cum_weight'] / total
    return df.merge(dedup[[q_col, out_col]], on=q_col).drop('cum_weight', axis=1)

nlss_data['poor'] = nlss_data.apply(
    lambda x: 0 if x['totcons_adj'] > x['pl'] else 1, axis=1)
nlss_data['ultrapoor'] = nlss_data.apply(
    lambda x: 0 if x['totcons_adj'] > (x['pl'] / 2) else 1, axis=1)
nlss_pls = nlss_hhs.merge(
    nlss_data[['hhid', 'poor', 'ultrapoor', 'log_con']], on='hhid')

poor_thresh = (nlss_pls['poor'] * nlss_pls['popw']).sum() \
    / nlss_pls['popw'].sum()
ultrapoor_thresh = (nlss_pls['ultrapoor'] * nlss_pls['popw']).sum() \
    / nlss_pls['popw'].sum()
print(poor_thresh, ultrapoor_thresh)

nlss_pls = get_weighted_quantile(nlss_pls, 'nlss_rwi', 'popw', 'nlss_rwi_q')
nlss_pls = nlss_pls.drop(['nlss_rwi', 'poor', 'ultrapoor', 'log_con'],
                         axis=1)

pop_reweight = states['pop'].sum() / nlss_pls['popw'].sum()

nlss_pls['asset_poor'] = nlss_pls['nlss_rwi_q']\
    .apply(lambda x: 0 if x > poor_thresh else 1)
nlss_pls['asset_ultrapoor'] = nlss_pls['nlss_rwi_q']\
    .apply(lambda x: 0 if x > ultrapoor_thresh else 1)

nlss_pls['asset_poor_w'] = nlss_pls.apply(
    lambda x: x['asset_poor'] * x['popw'] * pop_reweight, axis=1)
nlss_pls['asset_ultrapoor_w'] = nlss_pls.apply(
    lambda x: x['asset_ultrapoor'] * x['popw'] * pop_reweight, axis=1)

total_poor = states['pop'].sum() * poor_thresh
total_ultrapoor = states['pop'].sum() * ultrapoor_thresh
total_nonpoor = states['pop'].sum() * (1 - poor_thresh)


0.40532921993869897 0.08207280524503148


In [180]:
#@title Use only regions where all sources have data
roc_lgas = lgas[(lgas['lga'].isin(dhs_lga['lga'])) &
                (lgas['lga'].isin(nlss_lga['lga']))]
roc_wards = wards[(wards['ward'].isin(dhs_ward['ward'])) &
                  (wards['ward'].isin(nlss_ward['ward']))]


In [181]:
#@title Augment df helper
def augment_region_pl_df_uw(nlss_pls_reg):
    nlss_pls_reg['cum_pop'] = nlss_pls_reg['pop'].cumsum()

    nlss_pls_reg['asset_poor_w'] = nlss_pls_reg['pop'] * nlss_pls_reg['asset_poor']
    nlss_pls_reg['asset_ultrapoor_w'] = nlss_pls_reg['pop'] * nlss_pls_reg['asset_ultrapoor']

    nlss_pls_reg['asset_poor_inc'] = nlss_pls_reg['asset_poor_w'].cumsum() \
                                     / nlss_pls_reg['asset_poor_w'].sum()
    nlss_pls_reg['asset_ultrapoor_inc'] = nlss_pls_reg['asset_ultrapoor_w'].cumsum() \
                                         / nlss_pls_reg['asset_ultrapoor_w'].sum()
    nlss_pls_reg['asset_nonpoor_inc'] = (nlss_pls_reg['cum_pop'] -
                                         nlss_pls_reg['asset_poor_w'].cumsum()) \
                                         / (nlss_pls_reg['pop'].sum() -
                                            nlss_pls_reg['asset_poor_w'].sum())
    nlss_pls_reg['asset_nonultrapoor_inc'] = (nlss_pls_reg['cum_pop'] -
                                              nlss_pls_reg['asset_ultrapoor_w'].cumsum()) \
                                              / (nlss_pls_reg['pop'].sum() -
                                                 nlss_pls_reg['asset_ultrapoor_w'].sum())
    return nlss_pls_reg

In [182]:
#@title Get ROC helper
def get_roc(df):
    keep_cols = ['asset_nonpoor_inc', 'asset_poor_inc',
                 'asset_ultrapoor_inc', 'asset_nonultrapoor_inc']
    dummy_row = pd.Series(['dummy'] + [0] * (len(df.columns) - 1),
                          index=df.columns)
    df = df.append(dummy_row, ignore_index=True)

    df = df[keep_cols].sort_values('asset_nonpoor_inc')
    poor = auc(df['asset_nonpoor_inc'], df['asset_poor_inc'])
    df = df.sort_values('asset_nonultrapoor_inc')
    ultrapoor = auc(df['asset_nonultrapoor_inc'], df['asset_ultrapoor_inc'])
    print('poor', round(poor, 3), 'very poor', round(ultrapoor, 3))

In [183]:
#@title Correlation CI helpers
# http://faculty.washington.edu/gloftus/P317-318/Useful_Information/r_to_z/PearsonrCIs.pdf

def r_to_z(r):
    return 0.5 * np.log((1 + r) / (1 - r))

def z_to_r(z):
    return (np.exp(2 * z) - 1) / (np.exp(2 * z) + 1)

def get_r_ci(df, c1, c2, rounded=True):
    r, p = pearsonr(df[c1], df[c2])
    z = r_to_z(r)
    std = np.sqrt(1 / (len(df) - 3))
    if rounded:
        return round(r, 3), p, (round(z_to_r(z - 1.96 * std), 3), round(z_to_r(z + 1.96 * std), 3))
    return r, (p, z_to_r(z - 1.96 * std), z_to_r(z + 1.96 * std))

# Do mean targeting

In [194]:
#@title NLSS state level targeting DF

total_poor = states['pop'].sum() * poor_thresh
total_ultrapoor = states['pop'].sum() * ultrapoor_thresh
total_nonpoor = states['pop'].sum() * (1 - poor_thresh)

nlss_pls_state = nlss_pls.groupby('state')\
    [['nlss_rwi_w', 'asset_poor_w', 'asset_ultrapoor_w']].sum().reset_index()
nlss_pls_state = nlss_pls_state.merge(states)

nlss_pls_state['nlss_rwi'] = nlss_pls_state['nlss_rwi_w'] \
    / nlss_pls_state['pop']
nlss_pls_state = nlss_pls_state.sort_values(by='nlss_rwi')\
                               .drop('nlss_rwi_w', axis=1)
nlss_pls_state['cum_pop'] = nlss_pls_state['pop'].cumsum()

nlss_pls_state['asset_poor_inc'] = nlss_pls_state['asset_poor_w'].cumsum() \
    / total_poor
nlss_pls_state['asset_ultrapoor_inc'] = nlss_pls_state['asset_ultrapoor_w']\
    .cumsum() / total_ultrapoor
nlss_pls_state['asset_nonpoor_inc'] = (nlss_pls_state['cum_pop'] -
    nlss_pls_state['asset_poor_w'].cumsum()) \
    / (states['pop'].sum() * (1 - poor_thresh))
nlss_pls_state['asset_nonultrapoor_inc'] = (nlss_pls_state['cum_pop'] -
    nlss_pls_state['asset_ultrapoor_w'].cumsum()) \
    / (states['pop'].sum() * (1 - ultrapoor_thresh))


In [195]:
#@title LGA/ward poverty line df helper

# def get_region_pl_df_uw(pop_df, groupby_col, pls_df=nlss_pls):
#     nlss_pls_means = pls_df.groupby(groupby_col)[['asset_poor', 'asset_ultrapoor']]\
#                            .mean()\
#                            .reset_index()
#     nlss_pls_reg = pls_df.groupby(groupby_col)[['nlss_rwi_w', 'popw']]\
#                          .sum()\
#                          .reset_index()\
#                          .merge(pop_df[[groupby_col, 'pop']])\
#                          .merge(nlss_pls_means)
#     print(len(nlss_pls_reg))
#     nlss_pls_reg['nlss_rwi'] = nlss_pls_reg['nlss_rwi_w'] / nlss_pls_reg['popw']
#     nlss_pls_reg = nlss_pls_reg.sort_values(by='nlss_rwi')\
#                                .drop('nlss_rwi_w', axis=1)
#     return augment_region_pl_df_uw(nlss_pls_reg)
    
def get_region_pl_df_uw(pop_df, groupby_col, rwi_col_in='nlss_rwi_w',
                        rwi_col_out='nlss_rwi', pls_df=nlss_pls):
    nlss_pls_means = pls_df.groupby(groupby_col)[['asset_poor', 'asset_ultrapoor']]\
                           .mean()\
                           .reset_index()
    nlss_pls_reg = pls_df.groupby(groupby_col)[[rwi_col_in, 'popw']]\
                         .sum()\
                         .reset_index()\
                         .merge(pop_df[[groupby_col, 'pop']])\
                         .merge(nlss_pls_means)
    print(len(nlss_pls_reg))
    nlss_pls_reg[rwi_col_out] = nlss_pls_reg[rwi_col_in] / nlss_pls_reg['popw']
    nlss_pls_reg = nlss_pls_reg.sort_values(by=rwi_col_out)\
                               .drop(rwi_col_in, axis=1)
    return augment_region_pl_df_uw(nlss_pls_reg)
    



In [196]:
#@title NLSS LGA and ward level targeting DFs
# NLSS poverty lines, LGA
nlss_pls_lga = get_region_pl_df_uw(roc_lgas, 'lga')
print('Asset poor frac, lgas:', round(nlss_pls_lga['asset_poor_w'].sum() /
                                nlss_pls_lga['pop'].sum(), 3))
print('Asset very poor frac, lgas:', round(nlss_pls_lga['asset_ultrapoor_w'].sum() /
                                     nlss_pls_lga['pop'].sum(), 4))

# NLSS poverty lines, ward
nlss_pls_ward = get_region_pl_df_uw(roc_wards, 'ward')
print('Asset poor frac, wards:', round(nlss_pls_ward['asset_poor_w'].sum() /
                                nlss_pls_ward['pop'].sum(), 3))
print('Asset very poor frac, wards:', round(nlss_pls_ward['asset_ultrapoor_w'].sum() /
                                     nlss_pls_ward['pop'].sum(), 4))


597
Asset poor frac, lgas: 0.408
Asset very poor frac, lgas: 0.0912
464
Asset poor frac, wards: 0.29
Asset very poor frac, wards: 0.0662


In [197]:
# @title Get DIDL data for ROC plot

def get_inc_ex(pls_reg):
    pls_reg['cum_pop'] = pls_reg['pop'].cumsum()

    pls_reg['asset_poor_inc'] = pls_reg['asset_poor_w'].cumsum() \
                                / pls_reg['asset_poor_w'].sum()
    pls_reg['asset_ultrapoor_inc'] = pls_reg['asset_ultrapoor_w'].cumsum() \
                                     / pls_reg['asset_ultrapoor_w'].sum()
    pls_reg['asset_nonpoor_inc'] = (pls_reg['cum_pop'] - pls_reg['asset_poor_w'].cumsum()) \
                                    / (pls_reg['pop'].sum() - pls_reg['asset_poor_w'].sum())
    pls_reg['asset_nonultrapoor_inc'] = (pls_reg['cum_pop'] -
                                         pls_reg['asset_ultrapoor_w'].cumsum()) \
                                         / (pls_reg['pop'].sum() -
                                         pls_reg['asset_ultrapoor_w'].sum())
    return pls_reg

def get_didl_pl_df(groupby_col, didl_reg, nlss_pls_reg):
    didl_pls_reg = nlss_pls_reg.merge(didl_reg[[groupby_col, 'didl_orig_rwi']])\
                               .sort_values(by='didl_orig_rwi')
    return get_inc_ex(didl_pls_reg)
    

didl_pls_state = get_didl_pl_df('state', didl_orig_state, nlss_pls_state)
didl_pls_lga = get_didl_pl_df('lga', didl_orig_lga, nlss_pls_lga)
didl_pls_ward = get_didl_pl_df('ward', didl_orig_ward, nlss_pls_ward)


In [198]:
#@title Get DHS data for ROC plot

def get_dhs_pl_df(groupby_col, dhs_reg, nlss_pls_reg):
    dhs_pls_reg = nlss_pls_reg.merge(dhs_reg[[groupby_col, 'dhs_rwi']])\
                              .sort_values(by='dhs_rwi')
    return get_inc_ex(dhs_pls_reg)


common_lgas = dhs_lga.loc[dhs_lga['lga'].isin(nlss_lga['lga']), 'lga'].to_list()
nlss_dhs_pls = nlss_pls[nlss_pls['lga'].isin(common_lgas)]

dhs_pls_state = get_dhs_pl_df('state', dhs_state, nlss_pls_state)
dhs_pls_lga = get_dhs_pl_df('lga', dhs_lga, nlss_pls_lga)
dhs_pls_ward = get_dhs_pl_df('ward', dhs_ward, nlss_pls_ward)


In [199]:
#@title Make imputed DHS df -- LGAs
def impute_lga_with_state(row):
    drop_lga_df = dhs_eval.loc[(dhs_eval['lga'] != row['lga']) &
                               (dhs_eval['geo_state'] == row['state'])]
    return drop_lga_df['dhs_rwi_w'].sum() / drop_lga_df['weight'].sum()

dhs_eval['dhs_rwi_w'] = dhs_eval['weight'] * standardize(dhs_eval['dhs_rwi'])
dhs_lga_impute = dhs_lga.merge(lgas[['lga', 'state']]) #, how='right')

dhs_lga_impute['imputed_rwi'] = dhs_lga_impute.apply(impute_lga_with_state, axis=1)

print(round(pearsonr(dhs_lga_impute['imputed_rwi'], dhs_lga_impute['dhs_rwi'])[0], 3))


0.687


In [200]:
#@title Make imputed DHS df -- wards
def impute_ward_with_lga(row):
    drop_ward_df = dhs_eval.loc[(dhs_eval['ward'] != row['ward']) &
                                (dhs_eval['lga'] == row['lga'])]
    if len(drop_ward_df) == 0:
        return None
    return drop_ward_df['dhs_rwi'].sum() / drop_ward_df['weight'].count()

dhs_eval['dhs_rwi'] = standardize(dhs_eval['dhs_rwi'])
dhs_ward_impute = dhs_ward.merge(wards[['ward', 'lga']]) #, how='right')
# dhs_ward_impute = dhs_ward_impute[dhs_ward_impute['ward'].isin(nlss_ward['ward'])]

dhs_ward_impute['imputed_rwi'] = dhs_ward_impute.apply(impute_ward_with_lga, axis=1)

dhs_ward_impute = dhs_ward_impute.merge(dhs_lga_impute[['lga', 'imputed_rwi']].
                                        rename({'imputed_rwi': 'state_rwi'}, axis=1),
                                        on='lga')
print(len(dhs_ward_impute))
dhs_ward_impute.loc[dhs_ward_impute['imputed_rwi'].isna(), 'imputed_rwi'] = \
    dhs_ward_impute.loc[dhs_ward_impute['imputed_rwi'].isna(), 'state_rwi']

print(round(pearsonr(dhs_ward_impute['imputed_rwi'], dhs_ward_impute['dhs_rwi'])[0], 3))


1218
0.689


In [201]:
#@title Bootstrap LGAs w imputation, DHS
np.random.seed(123)

def get_random_roc(groupby_col, pop_df, n_to_replace, random_df,
                   nlss_pls, rwi_col='dhs_rwi'):
    replace = random_df.sample(n_to_replace)[groupby_col].to_list()
    random_df['rwi'] = random_df[rwi_col]
    random_df.loc[random_df[groupby_col].isin(replace), 'rwi'] = \
        random_df.loc[random_df[groupby_col].isin(replace), 'imputed_rwi']    
    dhs_pls_reg = nlss_pls.merge(random_df[[groupby_col, 'rwi']])\
                          .sort_values(by='rwi')
    
    return get_inc_ex(dhs_pls_reg)

def get_auc(df, col1, col2):
    return auc(df.sort_values(col1)[col1], df.sort_values(col1)[col2])

n_to_replace = round((1 - (len(dhs_lga_impute) / len(lgas))) * len(dhs_pls_lga))
dhs_lga_impute = dhs_lga_impute[dhs_lga_impute['lga'].isin(roc_lgas['lga'])]
print(n_to_replace, len(dhs_lga_impute))

lga_output = []
b = 1000
for i in range(b):
    lga_pls_i = get_random_roc('lga', lgas, n_to_replace, dhs_lga_impute, nlss_pls_lga)
    lga_output += [(get_auc(lga_pls_i, 'asset_nonpoor_inc', 'asset_poor_inc'),
                    get_auc(lga_pls_i, 'asset_nonultrapoor_inc', 'asset_ultrapoor_inc'),
                    lga_pls_i)]

noisy_dhs_lga_poor = sorted(lga_output, key=lambda x: x[0])[int(b / 2) - 1][2]
noisy_dhs_lga_ultrapoor = sorted(lga_output, key=lambda x: x[1])[int(b / 2) - 1][2]


110 597


In [202]:
#@title Bootstrap wards w imputation, DHS
np.random.seed(123)

n_to_replace = round((1 - (len(dhs_ward) / len(wards))) * len(dhs_pls_ward))
dhs_ward_impute = dhs_ward_impute[dhs_ward_impute['ward'].isin(roc_wards['ward'])]
print(n_to_replace, len(dhs_ward_impute))

ward_output = []
for i in range(b):
    ward_pls_i = get_random_roc('ward', wards, n_to_replace, dhs_ward_impute, nlss_pls_ward)
    ward_output += [(get_auc(ward_pls_i, 'asset_nonpoor_inc', 'asset_poor_inc'),
                     get_auc(ward_pls_i, 'asset_nonultrapoor_inc', 'asset_ultrapoor_inc'),
                     ward_pls_i)]

noisy_dhs_ward_poor = sorted(ward_output, key=lambda x: x[0])[int(b / 2) - 1][2]
noisy_dhs_ward_ultrapoor = sorted(ward_output, key=lambda x: x[1])[int(b / 2) - 1][2]


400 464


In [203]:
#@title Write csvs to make figures in R
imputed_wards = dhs_ward_impute[['ward', 'imputed_rwi']].merge(nlss_pls_ward[['ward', 'nlss_rwi']])
imputed_wards['imputed_rwi'] = standardize(imputed_wards['imputed_rwi'])
imputed_wards['nlss_rwi'] = standardize(imputed_wards['nlss_rwi'])
imputed_wards.to_csv('dhs_imputed_rwi_ward.csv', index=False)

imputed_lgas = dhs_lga_impute[['lga', 'imputed_rwi']].merge(nlss_pls_lga[['lga', 'nlss_rwi']])
imputed_lgas['imputed_rwi'] = standardize(imputed_lgas['imputed_rwi'])
imputed_lgas['nlss_rwi'] = standardize(imputed_lgas['nlss_rwi'])
imputed_lgas.to_csv('dhs_imputed_rwi_lga.csv', index=False)

pearsonr(imputed_lgas['imputed_rwi'], imputed_lgas['nlss_rwi'])

(0.646896384628388, 4.589551985764816e-72)

# Do fraction poor targeting

In [ ]:
#@title NLSS state level targeting DF, target by frac

# poor_thresh = (nlss_pls['poor'] * nlss_pls['popw']).sum() \
#     / nlss_pls['popw'].sum()
# ultrapoor_thresh = (nlss_pls['ultrapoor'] * nlss_pls['popw']).sum() \
#     / nlss_pls['popw'].sum()
# print(poor_thresh, ultrapoor_thresh)

total_poor = states['pop'].sum() * poor_thresh
total_ultrapoor = states['pop'].sum() * ultrapoor_thresh
total_nonpoor = states['pop'].sum() * (1 - poor_thresh)

nlss_pls_state = nlss_pls.groupby('state')\
    [['nlss_rwi_w', 'asset_poor_w', 'asset_ultrapoor_w']].sum().reset_index()
nlss_pls_state = nlss_pls_state.merge(states)

nlss_pls_state['frac_asset_poor'] = nlss_pls_state['asset_poor_w'] / \
    nlss_pls_state['pop']
nlss_pls_state['frac_asset_ultrapoor'] = nlss_pls_state['asset_ultrapoor_w'] / \
    nlss_pls_state['pop']

# Get errors of inc/ex for targeting based on % poor

def get_sort_by_frac_inc_ex(df, poverty_str, total, thresh):
    df = df.sort_values(by='frac_asset_%s' % poverty_str, ascending=False)
    df['cum_pop_sort_%s' % poverty_str] = df['pop'].cumsum()
    df['asset_%s_inc' % poverty_str] = df['asset_%s_w' % poverty_str].cumsum() \
        / total
    df['asset_non%s_inc' % poverty_str] = (df['cum_pop_sort_%s' % poverty_str] -
        df['asset_%s_w' % poverty_str].cumsum()) \
        / (states['pop'].sum() * (1 - thresh))
    return df

nlss_pls_state = get_sort_by_frac_inc_ex(nlss_pls_state, 'poor',
                                         total_poor, poor_thresh)
nlss_pls_state = get_sort_by_frac_inc_ex(nlss_pls_state, 'ultrapoor',
                                         total_ultrapoor, ultrapoor_thresh)


In [ ]:
#@title LGA/ward poverty line df helper, target by frac

def get_region_pl_df_frac(pop_df, groupby_col, pls_df=nlss_pls):
    nlss_pls_means = pls_df.groupby(groupby_col)[['asset_poor', 'asset_ultrapoor']]\
                            .mean()\
                            .reset_index()\
                            .merge(pop_df[[groupby_col, 'pop']])

    ultrapoor_cols = ['asset_ultrapoor_inc', 'asset_nonultrapoor_inc']
    poor_df = augment_region_pl_df_uw(
        nlss_pls_means.sort_values('asset_poor', ascending=False))\
        .drop(ultrapoor_cols, axis=1)
    ultrapoor_df = augment_region_pl_df_uw(
        nlss_pls_means.sort_values('asset_ultrapoor', ascending=False))
    return poor_df.merge(ultrapoor_df[[groupby_col] + ultrapoor_cols])


In [ ]:
#@title NLSS LGA and ward level targeting DFs, target by frac
# NLSS poverty lines, LGA
nlss_pls_lga = get_region_pl_df_frac(roc_lgas, 'lga')
print('Asset poor frac, lgas:', round(nlss_pls_lga['asset_poor_w'].sum() /
                                nlss_pls_lga['pop'].sum(), 3))
print('Asset very poor frac, lgas:', round(nlss_pls_lga['asset_ultrapoor_w'].sum() /
                                     nlss_pls_lga['pop'].sum(), 4))

# NLSS poverty lines, ward
nlss_pls_ward = get_region_pl_df_frac(roc_wards, 'ward')
print('Asset poor frac, wards:', round(nlss_pls_ward['asset_poor_w'].sum() /
                                nlss_pls_ward['pop'].sum(), 3))
print('Asset very poor frac, wards:', round(nlss_pls_ward['asset_ultrapoor_w'].sum() /
                                     nlss_pls_ward['pop'].sum(), 4))

len(nlss_pls_lga), len(nlss_pls_ward), len(roc_lgas)


Asset poor frac, lgas: 0.408
Asset very poor frac, lgas: 0.0912
Asset poor frac, wards: 0.29
Asset very poor frac, wards: 0.0662


(597, 464, 597)

In [ ]:
#@title Helper to estimate # poor per region from DIDL data
def n_poor(region, tiles, region_col, col_to_weight):
    def get_intersection_area(df):
            return df['geometry'].intersection(df['geometry_geom']).area

    overlaps = gpd.sjoin(region, tiles, how='right', op='intersects')\
                    .reset_index()\
                    .rename({'index': 'tile_index'}, axis=1)
    overlaps = overlaps.merge(region[['geometry', region_col]]\
                                .rename({'geometry': 'geometry_' + region_col}, axis=1),
                                on=region_col)
    edge_tiles = overlaps[overlaps.duplicated(subset=['tile_index'], keep=False)]
    center_tiles = overlaps.drop_duplicates(subset=['tile_index'], keep=False)

    edge_tiles['overlap_frac'] = edge_tiles.apply(lambda x: (x['geometry'].intersection(
                                                    x['geometry_' + region_col]).area /
                                                    x['geometry'].area),
                                                    axis=1)
    center_tiles['overlap_frac'] = 1
    overlaps = pd.concat([edge_tiles, center_tiles])

    overlaps['weighted_col'] = overlaps[col_to_weight] * overlaps['overlap_frac']

    return overlaps.groupby(region_col)[['weighted_col']]\
                        .sum()\
                        .reset_index()\
                        .rename({'weighted_col': col_to_weight}, axis=1)\
                        .merge(region[['geometry', region_col]], on=region_col)


In [ ]:
#@title Rank regions by fraction poor, DIDL

if False: # very slow -- save and re-use
    didl_tiles = get_weighted_quantile(didl_orig_all, 'estimated_rwi', 'pop', 'rwi_q')
    # didl_tiles = gpd.read_file(NOT_INC_DATA_PATH + 'estimates/didl_tile_wealth_quantiles.geojson')\
    #         .rename({'estimated_': 'estimated_rwi'}, axis=1)

    didl_tiles['poor'] = didl_tiles.apply(
        lambda x: x['pop'] if x['rwi_q'] < poor_thresh else 0, axis=1)
    didl_tiles['ultrapoor'] = didl_tiles.apply(
        lambda x: x['pop'] if x['rwi_q'] < ultrapoor_thresh else 0, axis=1)

def get_n_poor_region_df_didl(region_df, region_col):

    n_poor_df = n_poor(region_df[[region_col, 'geometry']], didl_tiles, region_col, 'poor')
    n_poor_df = n_poor_df.merge(region_df[[region_col, 'pop']])
    n_poor_df['frac_poor'] = n_poor_df['poor'] / n_poor_df['pop']

    n_ultrapoor_df = n_poor(region_df[[region_col, 'geometry']], didl_tiles,
                                region_col, 'ultrapoor')
    didl_orig_reg = n_poor_df.merge(n_ultrapoor_df[[region_col, 'ultrapoor']])
    didl_orig_reg['frac_ultrapoor'] = didl_orig_reg['ultrapoor'] \
        / didl_orig_reg['pop']
    return didl_orig_reg.drop(['poor', 'ultrapoor'], axis=1)

poor_frac_path = NOT_INC_DATA_PATH + 'generated_estimates/fraction/'
if False: # slow -- use saved data
    didl_orig_state = get_n_poor_region_df_didl(states, 'state')

    didl_orig_lga = get_n_poor_region_df_didl(lgas, 'lga')
    didl_orig_lga.drop('geometry', axis=1)\
                .to_csv(poor_frac_path + 'didl_lga_poor_frac.csv', index=False)

    didl_orig_ward = get_n_poor_region_df_didl(wards, 'ward')
    didl_orig_ward.drop('geometry', axis=1)\
                .to_csv(poor_frac_path + 'didl_ward_poor_frac.csv', index=False)

didl_orig_state = pd.read_csv(poor_frac_path + 'didl_state_poor_frac.csv')
didl_orig_lga = pd.read_csv(poor_frac_path + 'didl_lga_poor_frac.csv')
didl_orig_ward = pd.read_csv(poor_frac_path + 'didl_ward_poor_frac.csv')

In [ ]:
#@title Helper for poverty line dfs, DIDL & DHS, frac poor
def get_inc_ex_frac(pls_reg, target):
    pls_reg['cum_pop'] = pls_reg['pop'].cumsum()

    pls_reg['asset_%s_inc' % target] = \
        pls_reg['asset_%s_w' % target].cumsum() /\
        pls_reg['asset_%s_w' % target].sum()
    pls_reg['asset_non%s_inc' % target] = \
        (pls_reg['cum_pop'] - pls_reg['asset_%s_w' % target].cumsum()) /\
        (pls_reg['pop'].sum() - pls_reg['asset_%s_w' % target].sum())

    return pls_reg

def get_pl_df_frac_poor(groupby_col, didl_reg, nlss_pls_reg):
    sort_col = 'frac_poor'
    pls_poor = nlss_pls_reg.merge(didl_reg[[groupby_col, sort_col]])\
                           .sort_values(by=sort_col, ascending=False)
    pls_poor = get_inc_ex_frac(pls_poor, 'poor')

    sort_col = 'frac_ultrapoor'
    pls_ultrapoor = nlss_pls_reg.merge(didl_reg[[groupby_col, sort_col]])\
                                .sort_values(by=sort_col, ascending=False)
    pls_ultrapoor = get_inc_ex_frac(pls_ultrapoor, 'ultrapoor')

    return pls_poor[[groupby_col, 'asset_poor_w', 'pop', 'asset_poor_inc',
                     'asset_nonpoor_inc', 'frac_poor']].merge(
           pls_ultrapoor[[groupby_col, 'asset_ultrapoor_w',
                          'asset_ultrapoor_inc', 'asset_nonultrapoor_inc',
                          'frac_ultrapoor']])

In [ ]:
#@title Get DIDL data for ROC plot

pl_cols = ['asset_poor_w', 'asset_ultrapoor_w', 'pop']
didl_pls_state = get_pl_df_frac_poor('state', didl_orig_state, nlss_pls_state[['state'] + pl_cols])
didl_pls_lga = get_pl_df_frac_poor('lga', didl_orig_lga, nlss_pls_lga[['lga'] + pl_cols])
didl_pls_ward = get_pl_df_frac_poor('ward', didl_orig_ward, nlss_pls_ward[['ward'] + pl_cols])


In [ ]:
#@title Get DHS data for ROC plot
dhs_poor_frac = get_weighted_quantile(dhs_eval, 'dhs_rwi', 'weight', 'rwi_q')
dhs_poor_frac['poor'] = dhs_poor_frac.apply(
        lambda x: x['weight'] if x['rwi_q'] < poor_thresh else 0, axis=1)
dhs_poor_frac['ultrapoor'] = dhs_poor_frac.apply(
    lambda x: x['weight'] if x['rwi_q'] < ultrapoor_thresh else 0, axis=1)

def get_n_poor_region_df_dhs(poor_frac_df, region_col):
    df = poor_frac_df.groupby(region_col)[['poor', 'ultrapoor', 'weight']]\
                     .sum().reset_index()
    df['frac_poor'] = df['poor'] / df['weight']
    df['frac_ultrapoor'] = df['ultrapoor'] / df['weight']

    return df[[region_col, 'frac_poor', 'frac_ultrapoor']]

dhs_state = get_n_poor_region_df_dhs(dhs_poor_frac, 'geo_state').rename(
    {'geo_state': 'state'}, axis=1)
dhs_lga = get_n_poor_region_df_dhs(dhs_poor_frac, 'lga')
dhs_ward = get_n_poor_region_df_dhs(dhs_poor_frac, 'ward')

# dhs_pls_state = get_dhs_pl_df('state', dhs_state, nlss_pls_state)
# dhs_pls_lga = get_dhs_pl_df('lga', dhs_lga, nlss_pls_lga)
# dhs_pls_ward = get_dhs_pl_df('ward', dhs_ward, nlss_pls_ward)

dhs_pls_state = get_pl_df_frac_poor('state', dhs_state, nlss_pls_state[['state'] + pl_cols])
dhs_pls_lga = get_pl_df_frac_poor('lga', dhs_lga, nlss_pls_lga[['lga'] + pl_cols])
dhs_pls_ward = get_pl_df_frac_poor('ward', dhs_ward, nlss_pls_ward[['ward'] + pl_cols])


In [ ]:
# @title Make imputed DHS df -- LGAs, frac poor
def impute_lga_with_state(row, impute_col):
    drop_lga_df = dhs_poor_frac.loc[(dhs_poor_frac['lga'] != row['lga']) &
                                    (dhs_poor_frac['geo_state'] == row['state'])]
    return drop_lga_df[impute_col].sum() / drop_lga_df['weight'].sum()

dhs_lga_impute = dhs_lga.merge(lgas[['lga', 'state']])
dhs_lga_impute['imputed_frac_poor'] = dhs_lga_impute.apply(lambda x:
    impute_lga_with_state(x, 'poor'), axis=1)
dhs_lga_impute['imputed_frac_ultrapoor'] = dhs_lga_impute.apply(lambda x:
    impute_lga_with_state(x, 'ultrapoor'), axis=1)

print(round(pearsonr(dhs_lga_impute['imputed_frac_poor'],
                     dhs_lga_impute['frac_poor'])[0], 3))
print(round(pearsonr(dhs_lga_impute['imputed_frac_ultrapoor'],
                     dhs_lga_impute['frac_ultrapoor'])[0], 3))


0.684
0.61


In [ ]:
#@title Make imputed DHS df -- wards, frac poor
def impute_ward_with_lga(row, impute_col):
    drop_ward_df = dhs_poor_frac.loc[(dhs_poor_frac['ward'] != row['ward']) &
                                     (dhs_poor_frac['lga'] == row['lga'])]
    if len(drop_ward_df) == 0:
        return None

    return drop_ward_df[impute_col].sum() / drop_ward_df['weight'].sum()

dhs_ward_impute = dhs_ward.merge(wards[['ward', 'lga']])
dhs_ward_impute['imputed_frac_poor'] = dhs_ward_impute.apply(
    lambda x: impute_ward_with_lga(x, 'poor'), axis=1)
dhs_ward_impute['imputed_frac_ultrapoor'] = dhs_ward_impute.apply(
    lambda x: impute_ward_with_lga(x, 'ultrapoor'), axis=1)

dhs_ward_impute = dhs_ward_impute.merge(
    dhs_lga_impute[['lga', 'imputed_frac_poor', 'imputed_frac_ultrapoor']].
    rename({'imputed_frac_poor': 'state_frac_poor',
            'imputed_frac_ultrapoor': 'state_frac_ultrapoor',}, axis=1),
    on='lga')

def replace_with_state(df, colname):
    df.loc[df['imputed_%s' % colname].isna(), 'imputed_%s' % colname] = \
    df.loc[df['imputed_%s' % colname].isna(), 'state_%s' % colname]

replace_with_state(dhs_ward_impute, 'frac_poor')
replace_with_state(dhs_ward_impute, 'frac_ultrapoor')

print(round(pearsonr(dhs_ward_impute['imputed_frac_poor'],
                     dhs_ward_impute['frac_poor'])[0], 3))
print(round(pearsonr(dhs_ward_impute['imputed_frac_ultrapoor'],
                     dhs_ward_impute['frac_ultrapoor'])[0], 3))



0.632
0.503


In [ ]:
#@title Bootstrap helpers, frac poor
def get_random_roc(groupby_col, pop_df, n_to_replace, random_df, nlss_pls,
                   target):
    target_colname = 'target_frac_%s' % target

    replace = random_df.sample(n_to_replace)[groupby_col].to_list()
    random_df[target_colname] = random_df['frac_%s' % target]

    random_df.loc[random_df[groupby_col].isin(replace), target_colname] = \
        random_df.loc[random_df[groupby_col].isin(replace),
                      'imputed_frac_%s' % target]    
    dhs_pls_reg = nlss_pls.merge(random_df[[groupby_col, target_colname]])\
                          .sort_values(by=target_colname, ascending=False)
    return get_inc_ex_frac(dhs_pls_reg, target)

def get_auc(df, col1, col2):
    return auc(df.sort_values(col1)[col1], df.sort_values(col1)[col2])

In [ ]:
#@title Bootstrap LGAs w imputation, DHS
np.random.seed(123)

n_to_replace = round((1 - (len(dhs_lga_impute) / len(lgas))) * len(dhs_pls_lga))
dhs_lga_impute = dhs_lga_impute[dhs_lga_impute['lga'].isin(roc_lgas['lga'])]

lga_output = []   
b = 1000
for i in range(b):
    pl_cols = ['lga', 'asset_poor_w', 'asset_ultrapoor_w', 'pop']

    lga_pls_poor_i = get_random_roc('lga', lgas, n_to_replace, dhs_lga_impute,
                                    nlss_pls_lga[pl_cols], 'poor')
    lga_pls_ultrapoor_i = get_random_roc('lga', lgas, n_to_replace, dhs_lga_impute,
                                        nlss_pls_lga[pl_cols], 'ultrapoor')
    lga_pls_i = lga_pls_poor_i.merge(
        lga_pls_ultrapoor_i[['lga', 'asset_ultrapoor_inc', 'asset_nonultrapoor_inc']])

    lga_output += [(get_auc(lga_pls_i, 'asset_nonpoor_inc', 'asset_poor_inc'),
                    get_auc(lga_pls_i, 'asset_nonultrapoor_inc', 'asset_ultrapoor_inc'),
                    lga_pls_i)]

noisy_dhs_lga_poor = sorted(lga_output, key=lambda x: x[0])[int(b / 2) - 1][2]
noisy_dhs_lga_ultrapoor = sorted(lga_output, key=lambda x: x[1])[int(b / 2) - 1][2]

ultrapoor_cols = ['asset_nonultrapoor_inc', 'asset_ultrapoor_inc']
noisy_dhs_lga = noisy_dhs_lga_poor.drop(ultrapoor_cols, axis=1)\
    .merge(noisy_dhs_lga_ultrapoor[ultrapoor_cols + ['lga']])


In [ ]:
#@title Bootstrap wards w imputation, DHS
np.random.seed(123)

n_to_replace = round((1 - (len(dhs_ward) / len(wards))) * len(dhs_pls_ward))
dhs_ward_impute = dhs_ward_impute[dhs_ward_impute['ward'].isin(roc_wards['ward'])]
print(n_to_replace, len(dhs_ward_impute))

ward_output = []
for i in range(b):
    pl_cols = ['ward', 'asset_poor_w', 'asset_ultrapoor_w', 'pop']

    ward_pls_poor_i = get_random_roc('ward', wards, n_to_replace, dhs_ward_impute,
                                    nlss_pls_ward[pl_cols], 'poor')
    ward_pls_ultrapoor_i = get_random_roc('ward', wards, n_to_replace, dhs_ward_impute,
                                        nlss_pls_ward[pl_cols], 'ultrapoor')
    ward_pls_i = ward_pls_poor_i.merge(
        ward_pls_ultrapoor_i[['ward', 'asset_ultrapoor_inc', 'asset_nonultrapoor_inc']])

    ward_output += [(get_auc(ward_pls_i, 'asset_nonpoor_inc', 'asset_poor_inc'),
                    get_auc(ward_pls_i, 'asset_nonultrapoor_inc', 'asset_ultrapoor_inc'),
                    ward_pls_i)]

noisy_dhs_ward_poor = sorted(ward_output, key=lambda x: x[0])[int(b / 2) - 1][2]
noisy_dhs_ward_ultrapoor = sorted(ward_output, key=lambda x: x[1])[int(b / 2) - 1][2]

ultrapoor_cols = ['asset_nonultrapoor_inc', 'asset_ultrapoor_inc']
noisy_dhs_ward = noisy_dhs_ward_poor.drop(ultrapoor_cols, axis=1)\
    .merge(noisy_dhs_ward_ultrapoor[ultrapoor_cols + ['ward']])


400 464


In [ ]:
# @title Get ROC
print('NLSS')
get_roc(nlss_pls_state)
get_roc(nlss_pls_lga)
get_roc(nlss_pls_ward)
print('\nSatellite')
get_roc(didl_pls_state)
get_roc(didl_pls_lga)
get_roc(didl_pls_ward)
print('\nDHS upper')
get_roc(dhs_pls_state)
get_roc(dhs_pls_lga)
get_roc(dhs_pls_ward)
print('\nDHS lower')
get_roc(noisy_dhs_lga)
get_roc(noisy_dhs_ward)


NLSS
poor 0.789 very poor 0.769
poor 0.896 very poor 0.894
poor 0.95 very poor 0.964

Satellite
poor 0.769 very poor 0.72
poor 0.824 very poor 0.774
poor 0.861 very poor 0.835

DHS upper
poor 0.775 very poor 0.735
poor 0.812 very poor 0.713
poor 0.861 very poor 0.72

DHS lower
poor 0.804 very poor 0.721
poor 0.802 very poor 0.774


In [ ]:
#@title Get ward data for R
keep_cols = ['asset_nonpoor_inc', 'asset_poor_inc',
             'asset_ultrapoor_inc', 'asset_nonultrapoor_inc']
r_ward_nlss = nlss_pls_ward[keep_cols]
r_ward_didl = didl_pls_ward[keep_cols]
r_ward_dhs = dhs_pls_ward[keep_cols]
r_noisy_dhs_ward_poor = noisy_dhs_ward[keep_cols]\
    .sort_values('asset_nonpoor_inc')
r_noisy_dhs_ward_ultrapoor = noisy_dhs_ward[keep_cols]\
    .sort_values('asset_nonultrapoor_inc')

r_ward_nlss['type'] = 'Optimal'
r_ward_didl['type'] = 'Satellite'
r_ward_dhs['type'] = 'DHS'
r_noisy_dhs_ward_poor['type'] = 'DHS 86.3% imputed'
r_noisy_dhs_ward_ultrapoor['type'] = 'DHS 86.3% imputed'

r_ward_roc_df_poor = pd.concat([r_ward_nlss, r_ward_didl, r_ward_dhs,
                                r_noisy_dhs_ward_poor])
# r_ward_roc_df_poor.to_csv(OUT_DIR + 'ward_roc_poor_frac.csv', index=False)
r_ward_roc_df_ultrapoor = pd.concat([r_ward_nlss, r_ward_didl, r_ward_dhs,
                                     r_noisy_dhs_ward_ultrapoor])
# r_ward_roc_df_ultrapoor.to_csv(OUT_DIR + 'ward_roc_ultrapoor_frac.csv', index=False)


#Do median targeting

In [ ]:
#@title Median wealth helpers
nlss_pls_median = nlss_pls.copy()
nlss_pls_median['rwi'] = nlss_pls_median['nlss_rwi_w'] / nlss_pls_median['popw']
nlss_pls_med_state = nlss_pls_median.groupby('state')\
    .apply(get_weighted_quantile_by_group, 'rwi', 'popw', 'state_rwi_q')\
    .drop('state', axis=1)\
    .reset_index()\
    .drop('level_1', axis=1)

def get_weighted_quantile_by_group(df, sort_col, weight_col, out_col):
    df = df.sort_values(sort_col)
    df['cum_weight'] = df[weight_col].cumsum()
    total = df[weight_col].sum()
    dedup = df.drop_duplicates(sort_col, keep='last')
    dedup[out_col] = dedup['cum_weight'] / total
    return df.merge(dedup[[sort_col, out_col]], on=sort_col)\
             .drop('cum_weight', axis=1)

def get_median_rows(df, op, med_df=nlss_pls_med_state):
    df_state = df.groupby('state')['state_rwi_q']\
                 .apply(op).reset_index()\
                 .merge(med_df) 
    df_state['rwi'] = df_state['nlss_rwi_w'] / df_state['popw']
    return df_state[['state', 'state_rwi_q', 'rwi']].drop_duplicates()

def calc_median(x):
    if x['lower_q'] == 0.5:
        return x['lower_rwi']
    lower_weight = 1 / (0.5 - x['lower_q'])
    upper_weight = 1 / (x['upper_q'] - 0.5)
    return ((lower_weight * x['lower_rwi'] + upper_weight * x['upper_rwi']) /
            (lower_weight + upper_weight))


In [ ]:
#@title Get region-level NLSS median wealth estimates

nlss_pls_median = nlss_pls.copy()
nlss_pls_median['rwi'] = nlss_pls_median['nlss_rwi_w'] / nlss_pls_median['popw']
nlss_pls_med_state = nlss_pls_median.groupby('state')\
    .apply(get_weighted_quantile_by_group, 'rwi', 'popw', 'state_rwi_q')\
    .drop('state', axis=1)\
    .reset_index()\
    .drop('level_1', axis=1)

lower_bound_pls_state = get_median_rows(
    nlss_pls_med_state[nlss_pls_med_state['state_rwi_q'] <= 0.5],
    lambda x: max(x))\
    .rename({'state_rwi_q': 'lower_q', 'rwi': 'lower_rwi'}, axis=1)

upper_bound_pls_state = get_median_rows(
    nlss_pls_med_state[nlss_pls_med_state['state_rwi_q'] >= 0.5],
    lambda x: min(x))\
    .rename({'state_rwi_q': 'upper_q', 'rwi': 'upper_rwi'}, axis=1)

nlss_pls_med_state = lower_bound_pls_state.merge(upper_bound_pls_state,
                                                 on='state')
nlss_pls_med_state['median_rwi'] = nlss_pls_med_state.apply(calc_median, axis=1)

nlss_pls_med_lga = nlss_pls_median.groupby('lga')['rwi']\
                                  .median().reset_index()
nlss_pls_med_ward = nlss_pls_median.groupby('ward')['rwi']\
                                   .median().reset_index()


In [ ]:
#@title Make NLSS state-level targeting df
nlss_pls_state = nlss_pls.groupby('state')\
    [['asset_poor_w', 'asset_ultrapoor_w']].sum().reset_index()
nlss_pls_state = nlss_pls_state.merge(states)\
    .merge(nlss_pls_med_state[['state', 'median_rwi']])

nlss_pls_state = nlss_pls_state.sort_values(by='median_rwi')
nlss_pls_state['cum_pop'] = nlss_pls_state['pop'].cumsum()

nlss_pls_state['asset_poor_inc'] = nlss_pls_state['asset_poor_w'].cumsum() \
    / total_poor
nlss_pls_state['asset_ultrapoor_inc'] = nlss_pls_state['asset_ultrapoor_w']\
    .cumsum() / total_ultrapoor
nlss_pls_state['asset_nonpoor_inc'] = (nlss_pls_state['cum_pop'] -
    nlss_pls_state['asset_poor_w'].cumsum()) \
    / (states['pop'].sum() * (1 - poor_thresh))
nlss_pls_state['asset_nonultrapoor_inc'] = (nlss_pls_state['cum_pop'] -
    nlss_pls_state['asset_ultrapoor_w'].cumsum()) \
    / (states['pop'].sum() * (1 - ultrapoor_thresh))


In [ ]:
#@title LGA/ward poverty line df helper
def get_region_pl_df_uw(pop_df, groupby_col, target_df, pls_df=nlss_pls):
    nlss_pls_means = pls_df.groupby(groupby_col)[['asset_poor', 'asset_ultrapoor']]\
                           .mean()\
                           .reset_index()\
                           .merge(target_df)\
                           .merge(pop_df[[groupby_col, 'pop']])
    return augment_region_pl_df_uw(nlss_pls_means.sort_values(by='rwi'))
    


In [ ]:
#@title NLSS LGA and ward level targeting DFs
# NLSS poverty lines, LGA
nlss_pls_lga = get_region_pl_df_uw(roc_lgas, 'lga', nlss_pls_med_lga)
print('Asset poor frac, lgas:', round(nlss_pls_lga['asset_poor_w'].sum() /
                                nlss_pls_lga['pop'].sum(), 3))
print('Asset very poor frac, lgas:', round(nlss_pls_lga['asset_ultrapoor_w'].sum() /
                                     nlss_pls_lga['pop'].sum(), 4))

# NLSS poverty lines, ward
nlss_pls_ward = get_region_pl_df_uw(roc_wards, 'ward', nlss_pls_med_ward)
print('Asset poor frac, wards:', round(nlss_pls_ward['asset_poor_w'].sum() /
                                nlss_pls_ward['pop'].sum(), 3))
print('Asset very poor frac, wards:', round(nlss_pls_ward['asset_ultrapoor_w'].sum() /
                                     nlss_pls_ward['pop'].sum(), 4))


Asset poor frac, lgas: 0.408
Asset very poor frac, lgas: 0.0912
Asset poor frac, wards: 0.29
Asset very poor frac, wards: 0.0662


In [ ]:
#@title DIDL and DHS shared median helpers
def get_median_rows_didl(df, op, groupby, q_col, q_df, rwi_col):
    df_med = df.groupby(groupby)[q_col].apply(op).reset_index().merge(q_df) 
    return df_med[[groupby, q_col, rwi_col]].drop_duplicates()

def get_medians_from_quantiles(q_df, groupby_col, q_col, rwi_col):
    lower_bound = get_median_rows_didl(q_df[q_df[q_col] <= 0.5],
        lambda x: max(x), groupby_col, q_col, q_df, rwi_col)\
        .rename({q_col: 'lower_q', rwi_col: 'lower_rwi'}, axis=1)

    upper_bound = get_median_rows_didl(q_df[q_df[q_col] >= 0.5],
        lambda x: min(x), groupby_col, q_col, q_df, rwi_col)\
        .rename({q_col: 'upper_q', rwi_col: 'upper_rwi'}, axis=1)

    medians = lower_bound.merge(upper_bound, on=groupby_col)
    medians['median_rwi'] = medians.apply(calc_median, axis=1)
    return medians

def get_inc_ex(pls_reg):
    pls_reg['cum_pop'] = pls_reg['pop'].cumsum()

    pls_reg['asset_poor_inc'] = pls_reg['asset_poor_w'].cumsum() \
                                / pls_reg['asset_poor_w'].sum()
    pls_reg['asset_ultrapoor_inc'] = pls_reg['asset_ultrapoor_w'].cumsum() \
                                     / pls_reg['asset_ultrapoor_w'].sum()
    pls_reg['asset_nonpoor_inc'] = (pls_reg['cum_pop'] - pls_reg['asset_poor_w'].cumsum()) \
                                    / (pls_reg['pop'].sum() - pls_reg['asset_poor_w'].sum())
    pls_reg['asset_nonultrapoor_inc'] = (pls_reg['cum_pop'] -
                                         pls_reg['asset_ultrapoor_w'].cumsum()) \
                                         / (pls_reg['pop'].sum() -
                                         pls_reg['asset_ultrapoor_w'].sum())
    return pls_reg

def get_didl_dhs_pl_df(groupby_col, medians_reg, nlss_pls_reg, sort_col):
    pls_reg = nlss_pls_reg.merge(medians_reg[[groupby_col, sort_col]])\
                               .sort_values(by=sort_col)
    return get_inc_ex(pls_reg)

In [ ]:
#@title Get DIDL overlap with regions dataframe

def calc_overlaps(df, region):
    edge_tiles = df[df.duplicated(subset=['x', 'y'], keep=False)]
    center_tiles = df.drop_duplicates(subset=['x', 'y'], keep=False)

    def frac_intersecting(x):
        return (x['geometry'].intersection(x[region + '_geom']).area /
                x['geometry'].area)

    edge_tiles['overlap_frac'] = edge_tiles.apply(frac_intersecting, axis=1)
    center_tiles['overlap_frac'] = 1
    return pd.concat([edge_tiles, center_tiles])

def get_overlaps_df(didl_orig_all, region_df, region):
    didl_reg = gpd.sjoin(didl_orig_all, region_df[[region, 'geometry']],
                         op='intersects')\
               .merge(region_df[[region, 'geometry']]
               .rename({'geometry': region + '_geom'}, axis=1))\
               .drop(['index_right'], axis=1)
    overlaps = calc_overlaps(didl_reg, region)
    overlaps['region_pop'] = overlaps['pop'] * overlaps['overlap_frac']
    return overlaps

# ward_overlaps = get_overlaps_df(didl_orig_all, wards, 'ward')
# lga_overlaps = get_overlaps_df(didl_orig_all, lgas, 'lga')
# state_overlaps = get_overlaps_df(didl_orig_all, states, 'state')


In [ ]:
#@title Get DIDL ward/LGA median rwi

t0 = time.time()
median_path = NOT_INC_DATA_PATH + 'estimates/'

# didl_q_ward = ward_overlaps.groupby('ward').apply(
#     get_weighted_quantile_by_group, 'estimated_rwi', 'region_pop', 'rwi_q')
# didl_pls_med_ward = get_medians_from_quantiles(
#     didl_q_ward.drop(['ward'], axis=1).reset_index(),
#     'ward', 'rwi_q', 'estimated_rwi')
didl_pls_med_ward = pd.read_csv(median_path + 'didl_ward_median_rwi.csv')

# didl_q_lga = lga_overlaps.groupby('lga').apply(
#     get_weighted_quantile_by_group, 'estimated_rwi', 'region_pop', 'rwi_q')
# didl_pls_med_lga = get_medians_from_quantiles(
#     didl_q_lga.drop(['lga'], axis=1).reset_index(),
#     'lga', 'rwi_q', 'estimated_rwi')
didl_pls_med_lga = pd.read_csv(median_path + 'didl_lga_median_rwi.csv')

# didl_q_state = state_overlaps.groupby('state').apply(
#     get_weighted_quantile_by_group, 'estimated_rwi', 'region_pop', 'rwi_q')
# didl_pls_med_state = get_medians_from_quantiles(
#     didl_q_state.drop(['state'], axis=1).reset_index(),
#     'state', 'rwi_q', 'estimated_rwi')
didl_pls_med_state = pd.read_csv(median_path + 'didl_state_median_rwi.csv')



In [ ]:
#@title Get DIDL data for ROC plot

didl_pls_state = get_didl_dhs_pl_df('state', didl_pls_med_state,
                                nlss_pls_state.drop('median_rwi', axis=1),
                                'median_rwi')
didl_pls_lga = get_didl_dhs_pl_df('lga', didl_pls_med_lga,
                              nlss_pls_lga, 'median_rwi')
didl_pls_ward = get_didl_dhs_pl_df('ward', didl_pls_med_ward,
                               nlss_pls_ward, 'median_rwi')


In [ ]:
#@title Get DHS data for ROC plot

# DHS state
dhs_q_state = dhs_eval.groupby('geo_state').apply(
    get_weighted_quantile_by_group, 'dhs_rwi', 'weight', 'rwi_q')\
    .drop('geo_state', axis=1).reset_index()\
    .rename({'geo_state': 'state'}, axis=1)
dhs_pls_med_state = get_medians_from_quantiles(dhs_q_state, 'state',
                                               'rwi_q', 'dhs_rwi')
dhs_pls_state = get_didl_dhs_pl_df('state', dhs_pls_med_state,
    nlss_pls_state.drop('median_rwi', axis=1), 'median_rwi')

# DHS LGA
dhs_q_lga = dhs_eval.groupby('lga').apply(
    get_weighted_quantile_by_group, 'dhs_rwi', 'weight', 'rwi_q')\
    .drop('lga', axis=1).reset_index()
dhs_pls_med_lga = get_medians_from_quantiles(dhs_q_lga, 'lga',
                                             'rwi_q', 'dhs_rwi')
dhs_pls_lga = get_didl_dhs_pl_df('lga', dhs_pls_med_lga,
                                 nlss_pls_lga, 'median_rwi')

# DHS ward
dhs_q_ward = dhs_eval.groupby('ward').apply(
    get_weighted_quantile_by_group, 'dhs_rwi', 'weight', 'rwi_q')\
    .drop('ward', axis=1).reset_index()
dhs_pls_med_ward = get_medians_from_quantiles(dhs_q_ward, 'ward',
                                              'rwi_q', 'dhs_rwi')
dhs_pls_ward = get_didl_dhs_pl_df('ward', dhs_pls_med_ward,
                                  nlss_pls_ward, 'median_rwi')



In [ ]:
#@title Make imputed DHS df -- LGAs, median

def impute_dhs_median(df):
    df = get_weighted_quantile_by_group(df, 'dhs_rwi', 'weight', 'rwi_q')

    lower_q = df.loc[df['rwi_q'] <= 0.5, 'rwi_q'].max()
    lower_rwi = df.loc[df['rwi_q'] == lower_q, 'dhs_rwi'].iloc[0]
    upper_q = df.loc[df['rwi_q'] >= 0.5, 'rwi_q'].min()
    upper_rwi = df.loc[df['rwi_q'] == upper_q, 'dhs_rwi'].iloc[0]

    if lower_rwi == upper_rwi:
        return lower_rwi
    else: 
        lower_weight = 1 / (0.5 - lower_q)
        upper_weight = 1 / (upper_q - 0.5)
        return ((lower_weight * lower_rwi + upper_weight * upper_rwi) /
                (lower_weight + upper_weight))

def impute_lga_med_with_state(row):
    drop_lga_df = dhs_eval.loc[(dhs_eval['lga'] != row['lga']) &
                            (dhs_eval['geo_state'] == row['state'])]
    return impute_dhs_median(drop_lga_df)

dhs_lga_impute = dhs_pls_med_lga.merge(lgas[['lga', 'state']])
dhs_lga_impute['imputed_median'] = dhs_lga_impute.apply(
    impute_lga_med_with_state, axis=1)

print(round(pearsonr(dhs_lga_impute['median_rwi'],
                     dhs_lga_impute['imputed_median'])[0], 3))



0.687


In [ ]:
# @title Make imputed DHS df -- wards, median
t0 = time.time()
def impute_ward_with_lga(row):
    drop_ward_df = dhs_eval.loc[(dhs_eval['ward'] != row['ward']) &
                                 (dhs_eval['lga'] == row['lga'])]
    if len(drop_ward_df) == 0:
        return None
    return impute_dhs_median(drop_ward_df)

dhs_ward_impute = dhs_pls_med_ward.merge(wards[['ward', 'lga']])
dhs_ward_impute['imputed_median'] = dhs_ward_impute.apply(
    impute_ward_with_lga, axis=1)
    
dhs_ward_impute = dhs_ward_impute.merge(
    dhs_lga_impute[['lga', 'imputed_median']].
    rename({'imputed_median': 'state_median'}, axis=1),
    on='lga')

def replace_with_state(df, colname):
    df.loc[df['imputed_%s' % colname].isna(), 'imputed_%s' % colname] = \
    df.loc[df['imputed_%s' % colname].isna(), 'state_%s' % colname]

replace_with_state(dhs_ward_impute, 'median')
print(round(pearsonr(dhs_ward_impute['imputed_median'],
                     dhs_ward_impute['median_rwi'])[0], 3))


0.685


In [ ]:
#@title Bootstrap LGAs w imputation, DHS
np.random.seed(123)

def get_random_roc(groupby_col, pop_df, n_to_replace, random_df, nlss_pls):
    replace = random_df.sample(n_to_replace)[groupby_col].to_list()
    random_df['rwi'] = random_df['median_rwi']
    random_df.loc[random_df[groupby_col].isin(replace), 'rwi'] = \
        random_df.loc[random_df[groupby_col].isin(replace), 'imputed_median']    
    dhs_pls_reg = nlss_pls.merge(random_df[[groupby_col, 'rwi']])\
                          .sort_values(by='rwi')
    return get_inc_ex(dhs_pls_reg)

def get_auc(df, col1, col2):
    return auc(df.sort_values(col1)[col1], df.sort_values(col1)[col2])

n_to_replace = round((1 - (len(dhs_lga_impute) / len(lgas))) * len(dhs_pls_lga))
dhs_lga_impute = dhs_lga_impute[dhs_lga_impute['lga'].isin(roc_lgas['lga'])]
print(n_to_replace, len(dhs_lga_impute))

nlss_pls_cols = ['lga', 'pop', 'asset_poor_w', 'asset_ultrapoor_w',
                 'asset_poor_inc', 'asset_ultrapoor_inc', 'asset_nonpoor_inc',
                 'asset_nonultrapoor_inc']
nlss_pls_impute = nlss_pls_lga[nlss_pls_cols]

lga_output = []
b = 1000

for i in range(b):
    lga_pls_i = get_random_roc('lga', lgas, n_to_replace,
                               dhs_lga_impute, nlss_pls_impute)
    lga_output += [(get_auc(lga_pls_i, 'asset_nonpoor_inc', 'asset_poor_inc'),
        get_auc(lga_pls_i, 'asset_nonultrapoor_inc', 'asset_ultrapoor_inc'),
        lga_pls_i)]

noisy_dhs_lga_poor = sorted(lga_output, key=lambda x: x[0])[int(b / 2) - 1][2]
noisy_dhs_lga_ultrapoor = sorted(lga_output, key=lambda x: x[1])[int(b / 2) - 1][2]


110 597


In [ ]:
#@title Bootstrap wards w imputation, DHS
np.random.seed(123)

n_to_replace = round((1 - (len(dhs_ward) / len(wards))) * len(dhs_pls_ward))
dhs_ward_impute = dhs_ward_impute[dhs_ward_impute['ward'].isin(roc_wards['ward'])]
print(n_to_replace, len(dhs_ward_impute))

nlss_pls_cols = ['ward', 'pop', 'asset_poor_w', 'asset_ultrapoor_w',
                 'asset_poor_inc', 'asset_ultrapoor_inc', 'asset_nonpoor_inc',
                 'asset_nonultrapoor_inc']
nlss_pls_impute = nlss_pls_ward[nlss_pls_cols]

ward_output = []
for i in range(b):
    ward_pls_i = get_random_roc('ward', wards, n_to_replace,
                                dhs_ward_impute, nlss_pls_impute)
    ward_output += [(get_auc(ward_pls_i, 'asset_nonpoor_inc', 'asset_poor_inc'),
        get_auc(ward_pls_i, 'asset_nonultrapoor_inc', 'asset_ultrapoor_inc'),
        ward_pls_i)]

noisy_dhs_ward_poor = sorted(ward_output, key=lambda x: x[0])[int(b / 2) - 1][2]
noisy_dhs_ward_ultrapoor = sorted(ward_output, key=lambda x: x[1])[int(b / 2) - 1][2]


400 464


In [ ]:
#@title Get ward data for R
keep_cols = ['asset_nonpoor_inc', 'asset_poor_inc',
             'asset_ultrapoor_inc', 'asset_nonultrapoor_inc']
r_ward_nlss = nlss_pls_ward[keep_cols]
r_ward_didl = didl_pls_ward[keep_cols]
# r_ward_dhs = dhs_pls_ward[keep_cols]
r_noisy_dhs_ward_poor = noisy_dhs_ward_poor[keep_cols]\
    .sort_values('asset_nonpoor_inc')
r_noisy_dhs_ward_ultrapoor = noisy_dhs_ward_ultrapoor[keep_cols]\
    .sort_values('asset_nonultrapoor_inc')

r_ward_nlss['type'] = 'Optimal'
r_ward_didl['type'] = 'Satellite'
# r_ward_dhs['type'] = 'DHS'
r_noisy_dhs_ward_poor['type'] = 'DHS'
r_noisy_dhs_ward_ultrapoor['type'] = 'DHS'

r_ward_roc_df_poor = pd.concat([r_ward_nlss, r_ward_didl, #r_ward_dhs,
                                r_noisy_dhs_ward_poor])
# r_ward_roc_df_poor.to_csv(OUT_DIR + 'ward_roc_poor_median.csv', index=False)
r_ward_roc_df_ultrapoor = pd.concat([r_ward_nlss, r_ward_didl, #r_ward_dhs,
                                     r_noisy_dhs_ward_ultrapoor])
# r_ward_roc_df_ultrapoor.to_csv(OUT_DIR + 'ward_roc_ultrapoor_median.csv', index=False)

# frac_poor = pd.read_csv('drive/MyDrive/poverty_map_paper/output/figure_data/0721/ward_roc_ultrapoor_frac_old.csv')
# frac_poor = frac_poor[frac_poor['type'] != 'DHS']
# frac_poor.loc[frac_poor['type'] == 'DHS 86.3% imputed', 'type'] = 'DHS'
# frac_poor.to_csv('drive/MyDrive/poverty_map_paper/output/figure_data/0721/ward_roc_ultrapoor_frac.csv', index=False)

In [ ]:
#@title Get ROC
print('NLSS')
get_roc(nlss_pls_state)
get_roc(nlss_pls_lga)
get_roc(nlss_pls_ward)
print('\nSatellite')
get_roc(didl_pls_state)
get_roc(didl_pls_lga)
get_roc(didl_pls_ward)
print('\nDHS upper')
get_roc(dhs_pls_state)
get_roc(dhs_pls_lga)
get_roc(dhs_pls_ward)
print('\nDHS lower')
get_roc(noisy_dhs_lga_poor)
get_roc(noisy_dhs_lga_ultrapoor)
get_roc(noisy_dhs_ward_poor)
get_roc(noisy_dhs_ward_ultrapoor)


# Propensity score parity

In [204]:
#@title Get propensity score dataframe
"""
(1) Latitude -- hh_gps_latitude
(2) Longitude -- hh_gps_longitude
(3) Poverty -- nlss_hhs['nlss_rwi']
(4) Electrification -- electricity (ignore?)
(5) Population
    - urban/rural status of ward
    - pop density of ward
"""

keep_cols = ['cluster', 'hhid', 'hh_gps_latitude', 'hh_gps_longitude', 'ward']
propensity_df = nlss_data[keep_cols].merge(nlss_hhs[['hhid', 'nlss_rwi']])\
                                    .merge(wards[['ward', 'wardclass', 'geometry', 'pop']])

propensity_df['density'] = propensity_df.apply(lambda x: x['pop'] / x['geometry'].area, axis=1)
# propensity_df['female_hhh'] = propensity_df['male_hhh'].apply(lambda x: 0 if x == 'yes' else 1)
# propensity_df = propensity_df.drop('male_hhh', axis=1)

# propensity_df = propensity_df.groupby('cluster').mean().reset_index()

propensity_cols = ['hh_gps_latitude', 'hh_gps_longitude', 'nlss_rwi', 'density']
for col in propensity_cols:
    propensity_df[col + '_norm'] = standardize(propensity_df[col])
norm_cols = [x + '_norm' for x in propensity_cols]

propensity_df = propensity_df[propensity_df['ward'].isin(nlss_pls_ward['ward'])]


In [205]:
#@title Get demographic group dataframes
# Helper to get dummy cols by group type
def get_group_cols(col_df, in_colname, out_colnames, lambdas):
    p = propensity_df.merge(col_df[['hhid', in_colname]])
    for i in range(len(out_colnames)):
        p[out_colnames[i]] = p[in_colname].apply(lambdas[i])
    p.drop(in_colname, axis=1)
    return p

# Read in religion/language data
religion = pd.read_stata(NOT_INC_DATA_PATH + 'nlss/sect1_roster.dta')
religion = religion.loc[religion['s01q03'] == '1. HEAD', ['hhid', 's01q11']]\
                   .rename({'s01q11': 'religion'}, axis=1)
lang = pd.read_stata(NOT_INC_DATA_PATH + 'nlss/sect_result.dta')[['hhid', 'Lang_Resp']]\
       .rename({'Lang_Resp': 'language'}, axis=1)

# Get gender df
gender_out = ['male', 'female']
gender_lambdas = [lambda x: 0 if x == 'no' else 1,
                  lambda x: 0 if x == 'yes' else 1]
gender_df = get_group_cols(nlss_data, 'male_hhh', gender_out, gender_lambdas)

# Get age df
age_out = ['lt_30', '30_45', '45_60', 'gte_60']
age_lambdas = [lambda x: 1 if x < 30 else 0,
               lambda x: 1 if x >= 30 and x < 45 else 0,
               lambda x: 1 if x >= 45 and x < 60 else 0,
               lambda x: 1 if x >= 60 else 0]
age_df = get_group_cols(nlss_data, 'age_hhh', age_out, age_lambdas)

# Get religion df
relig_out = ['christian', 'islam', 'traditional', 'other']
relig_lambdas = [lambda x: 1 if x == '1. CHRISTIAN' else 0,
                 lambda x: 1 if x == '2. ISLAM' else 0,
                 lambda x: 1 if x == '3. TRADITIONAL' else 0,
                 lambda x: 1 if x == '4. OTHER' else 0]
relig_df = get_group_cols(religion, 'religion', relig_out, relig_lambdas)

# Get ethnic group parity data
lang_out = ['english', 'hausa', 'yoruba', 'igbo', 'other']
lang_lambdas = [lambda x: 1 if x == '1. ENGLISH' else 0,
                lambda x: 1 if x == '2. HAUSA' else 0,
                lambda x: 1 if x == '3. YORUBA' else 0,
                lambda x: 1 if x == '4. IGBO' else 0,
                lambda x: 1 if x == '5. OTHER SPECIFY' else 0]
lang_df = get_group_cols(lang.dropna(), 'language', lang_out, lang_lambdas) # one missing


In [206]:
#@title Counterfactual helpers
def add_pls(df):
    return df[['hhid', 'ward', 'cf_hhid', 'cf_ward']]\
        .merge(nlss_pls[['hhid', 'nlss_rwi_q']])\
        .rename({'nlss_rwi_q': 'rwi_q'}, axis=1)\
        .merge(nlss_pls[['hhid', 'nlss_rwi_q']],
            left_on='cf_hhid', right_on='hhid', suffixes=('', '_x'))\
        .drop(['hhid_x'], axis=1)\
        .rename({'nlss_rwi_q': 'cf_rwi_q'}, axis=1)

def add_targeting_data(df, target_df, suffix):
    df = df.merge(target_df[['ward', 'cum_pop']])\
           .rename({'cum_pop': 'target_q_' + suffix}, axis=1)\
           .merge(target_df[['ward', 'cum_pop']], left_on='cf_ward', right_on='ward',
                  suffixes=('', '_x'))\
           .drop('ward_x', axis=1)\
           .rename({'cum_pop': 'cf_target_q_' + suffix}, axis=1)
    df['target_q_' + suffix] /= target_df['cum_pop'].max()
    df['cf_target_q_' + suffix] /= target_df['cum_pop'].max()
    return df

def get_counterfactuals(df, get_cf_for):
    needs_cf = df[df[get_cf_for] == 1]
    cf_pool = df[df[get_cf_for] == 0]

    nn = NearestNeighbors(n_neighbors=1, metric='euclidean').fit(cf_pool[norm_cols])
    neighbors = nn.kneighbors(needs_cf[norm_cols], return_distance=False)
    needs_cf['cf_hhid'] = [cf_pool.iloc[x]['hhid'].iloc[0] for x in neighbors]
    needs_cf['cf_ward'] = [cf_pool.iloc[x]['ward'].iloc[0] for x in neighbors]

    cf = add_pls(needs_cf)
    cf = add_targeting_data(cf, nlss_pls_ward, 'nlss')
    cf = add_targeting_data(cf, dhs_pls_ward, 'dhs')
    return add_targeting_data(cf, didl_pls_ward, 'didl')

"""
Mean pov % (lower = poorer) - target % (lower = sooner)
> 0 implies hh is targeted earlier than it should be
< 0 implies hh is targeted later than it should be
"""
# def get_mean_residuals(df, delta_only=False):
#     group_resid, cf_resid, delta, pvalues = {}, {}, {}, {}
#     for suffix in ['nlss', 'dhs', 'didl']:
#         group_resid[suffix] = (df['rwi_q'] - df['target_q_' + suffix]).mean()
#         cf_resid[suffix] = (df['cf_rwi_q'] - df['cf_target_q_' + suffix]).mean()
#         delta[suffix] = group_resid[suffix] - cf_resid[suffix]

#     if delta_only:
#         return delta
#     return group_resid, cf_resid, delta

# def print_residuals(delta, group, group_df):
#     for k in delta:
#         print(group, k, 'N=%d' % len(group_df), round(delta[k], 3))

# def get_targeting_deltas_by_group(group_df, group_cols, verbose=False):
#     cfs = {}
#     residuals = {}
#     for col in group_cols:
#         cfs[col] = get_counterfactuals(group_df, col)
#         residuals[col] = get_mean_residuals(cfs[col])
#         if verbose:
#             print_residuals(residuals[col], col, cfs[col])
#     return cfs, residuals

def get_mean_residuals(df, colname):
    data = {'group': [], 'target_type': [], 'residual_type': [], 'residual': [], 'p': []}
    for suffix in ['nlss', 'dhs', 'didl']:
        data['group'] += [colname] * 3
        data['target_type'] += [suffix] * 3
        data['residual_type'] += ['group', 'counterfactual', 'delta']

        group_resid = df['rwi_q'] - df['target_q_' + suffix]
        cf_resid = df['cf_rwi_q'] - df['cf_target_q_' + suffix]

        data['residual'] += [group_resid.mean(), cf_resid.mean(), (group_resid - cf_resid).mean()]
        data['p'] += [ttest_1samp(group_resid, popmean=0).pvalue,
                    ttest_1samp(cf_resid, popmean=0).pvalue,
                    ttest_ind(group_resid, cf_resid).pvalue]

    return pd.DataFrame(data)

def get_targeting_deltas_by_group(group_df, group_cols):
    cfs = {}
    residuals = []
    for col in group_cols:
        cfs[col] = get_counterfactuals(group_df, col)
        residuals.append(get_mean_residuals(cfs[col], col))

    return cfs, pd.concat(residuals)


In [207]:
#@title Get barchart data
gender_cfs, gender_residuals = get_targeting_deltas_by_group(gender_df, gender_out)
age_cfs, age_residuals = get_targeting_deltas_by_group(age_df, age_out)
relig_cfs, relig_residuals = get_targeting_deltas_by_group(relig_df, relig_out)
lang_cfs, lang_residuals = get_targeting_deltas_by_group(lang_df, lang_out)

all_residuals = pd.concat([gender_residuals, age_residuals, relig_residuals, lang_residuals])

all_deltas = all_residuals[all_residuals['residual_type'] == 'delta'].dropna()
all_deltas[all_deltas['p'] < 0.05]

cf_barchart_data = all_deltas.rename({'group': 'var',
                                      'target_type': 'sort',
                                      'residual': 'under_over'}, axis=1)\
                             .drop('residual_type', axis=1)

# cf_barchart_data.to_csv(OUT_DIR + 'counterfactual_parity.csv', index=False)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3724: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


#Main parity results

Needs to be run using data from mean targeting

In [210]:
#@title Parity helpers
poor_cols = ['nonpoor', 'poor', 'ultrapoor']

def get_hh_groups(group_col, group_names, group_lambdas, hh_data):
    hh_groups = hh_data[['ward', 'lga', 'state', 'poor', 'ultrapoor', group_col]]
    for i in range(len(group_names)):
        hh_groups[group_names[i]] = hh_groups[group_col].apply(group_lambdas[i])

    hh_groups['nonpoor'] = hh_groups['poor'].apply(lambda x: 0 if x == 1 else 1)
    num_poor_cols = []

    for pc in poor_cols:
        for c in group_names:
            colname = '_'.join([pc, c])
            hh_groups[colname] = hh_groups[pc] * hh_groups[c]
            num_poor_cols.append(colname)
    
    return num_poor_cols, hh_groups

def choose_pls_df(region):
    assert region in set(['ward', 'lga'])
    if region == 'ward':
        return nlss_pls_ward[['ward', 'pop', 'nlss_rwi']]
    if region == 'lga':
        return nlss_pls_lga[['lga', 'pop', 'nlss_rwi']]
    
def get_region_groups(group_names, num_poor_cols, hh_groups, region, sample=False):
    pls_df = choose_pls_df(region)
    groups = hh_groups.groupby(region)\
                        [num_poor_cols + group_names + poor_cols]\
                        .sum().reset_index()\
                        .merge(pls_df)\
                        .sort_values('nlss_rwi')
    if sample:
        groups = groups.sample(frac=1, replace=True)
    groups['sample_size'] = groups[group_names].sum(axis=1)
    reweight = groups['pop'] / groups['sample_size']
    for col in group_names:
        groups[col] *= reweight
    return groups

def choose_dhs_df(region):
    assert region in set(['ward', 'lga'])
    if region == 'ward':
        return noisy_dhs_ward_poor[['ward', 'rwi']]
    if region == 'lga':
        return noisy_dhs_lga_poor[['lga', 'rwi']]

def choose_didl_df(region):
    assert region in set(['ward', 'lga'])
    if region == 'ward':
        return didl_orig_ward[['ward', 'didl_orig_rwi']]
    if region == 'lga':
        return didl_orig_lga[['lga', 'didl_orig_rwi']]

def get_plot_df(groups, group_names, region):
    dhs_rwi_df = choose_dhs_df(region).rename({'rwi': 'dhs_rwi'}, axis=1)
    target_df = groups[[region, 'pop', 'nlss_rwi'] + group_names]\
        .merge(dhs_rwi_df)\
        .merge(choose_didl_df(region))

    plot_df = pd.DataFrame()

    for rwi_col in ['nlss_rwi', 'didl_orig_rwi', 'dhs_rwi']:
        rwi_str = rwi_col.split('_')[0]
        target_df = target_df.sort_values(rwi_col)
        plot_df['frac_pop_' + rwi_str] = target_df['pop'].cumsum() / \
            target_df['pop'].sum()
        for group_col in group_names:
            plot_colname = 'frac_%s_%s' % (group_col, rwi_str)
            plot_df[plot_colname] = target_df[group_col].cumsum() / \
                target_df['pop'].cumsum()

    return plot_df

def get_frac_poor_by_group(groups, group_names):
    reweight = groups['pop'] / groups['sample_size']
    total_poor = (groups['poor'] * reweight).sum()

    frac_poor_in_group = {}
    for i in range(len(group_names)):
        group = group_names[i]
        frac_poor_in_group[group] = \
            (groups['poor_' + group] * reweight).sum() / total_poor
    
    return frac_poor_in_group

def get_parity_data(group_col, group_names, group_lambdas, hh_data=nlss_data,
                    region='ward', sample=False):
    num_poor_cols, hh_groups = \
        get_hh_groups(group_col, group_names, group_lambdas, hh_data)
    region_groups = get_region_groups(group_names, num_poor_cols, hh_groups, region, sample)
    plot_df = get_plot_df(region_groups, group_names, region)
    frac_poor_by_group = get_frac_poor_by_group(region_groups, group_names)
    for group in frac_poor_by_group:
        plot_df[group + '_poor'] = frac_poor_by_group[group]

    return plot_df




In [212]:
#@title Get parity data
# Get religion and ethnic group household data
religion = pd.read_stata(NOT_INC_DATA_PATH + 'nlss/sect1_roster.dta')
religion = religion.loc[religion['s01q03'] == '1. HEAD', ['hhid', 's01q11']]\
                   .rename({'s01q11': 'religion'}, axis=1)
lang = pd.read_stata(NOT_INC_DATA_PATH + 'nlss/sect_result.dta')[['hhid', 'Lang_Resp']]\
       .rename({'Lang_Resp': 'language'}, axis=1)
nlss_aug = nlss_data.merge(religion).merge(lang)

# Get gender parity data
gender_col = 'male_hhh'
gender_names = ['male', 'female']
gender_lambdas = [lambda x: 0 if x == 'no' else 1,
                 lambda x: 0 if x == 'yes' else 1]
gender_df = get_parity_data(gender_col, gender_names, gender_lambdas)
gender_df_lga = get_parity_data(gender_col, gender_names, gender_lambdas, region='lga')

# Get age parity data
age_col = 'age_hhh'
age_names = ['lt_30', '30_45', '45_60', 'gte_60']
age_lambdas = [lambda x: 1 if x < 30 else 0,
               lambda x: 1 if x >= 30 and x < 45 else 0,
               lambda x: 1 if x >= 45 and x < 60 else 0,
               lambda x: 1 if x >= 60 else 0]
age_df = get_parity_data(age_col, age_names, age_lambdas)
age_df_lga = get_parity_data(age_col, age_names, age_lambdas, region='lga')

# Get religion parity data
relig_col = 'religion'
relig_names = ['christian', 'islam', 'traditional', 'other']
relig_lambdas = [lambda x: 1 if x == '1. CHRISTIAN' else 0,
                 lambda x: 1 if x == '2. ISLAM' else 0,
                 lambda x: 1 if x == '3. TRADITIONAL' else 0,
                 lambda x: 1 if x == '4. OTHER' else 0]
relig_df = get_parity_data(relig_col, relig_names, relig_lambdas, nlss_aug)
relig_df_lga = get_parity_data(relig_col, relig_names, relig_lambdas, nlss_aug, region='lga')

# Get ethnic group parity data
lang_col = 'language'
lang_names = ['english', 'hausa', 'yoruba', 'igbo', 'other']
lang_lambdas = [lambda x: 1 if x == '1. ENGLISH' else 0,
                 lambda x: 1 if x == '2. HAUSA' else 0,
                 lambda x: 1 if x == '3. YORUBA' else 0,
                 lambda x: 1 if x == '4. IGBO' else 0,
                lambda x: 1 if x == '5. OTHER SPECIFY' else 0]
lang_df = get_parity_data(lang_col, lang_names, lang_lambdas,
                          nlss_aug[~nlss_aug['language'].isna()]) # one missing
lang_df_lga = get_parity_data(lang_col, lang_names, lang_lambdas,
                              nlss_aug[~nlss_aug['language'].isna()], region='lga') # one missing


In [213]:
#@title Get data for barchart
sort = 'nlss'
frac = 0.1
groups = ['male', 'female']

def get_barchart_data(df, groups, sort, demographic=None, frac=0.1):

    def get_under_over(x):
        actual = df[x['var'] + '_poor'].iloc[0]
        return (x['frac'] - actual) / actual * 100

    pop_col = 'frac_pop_' + sort
    target_cols = ['frac_%s_%s' % (g, sort) for g in groups]

    lower_pop = df.loc[df[pop_col] <= frac, pop_col].max()
    upper_pop = df.loc[df[pop_col] >= frac, pop_col].min()
    lower = df.loc[df[pop_col] == lower_pop, target_cols + [pop_col]]
    upper = df.loc[df[pop_col] == upper_pop, target_cols + [pop_col]]

    avg = pd.concat([lower, upper])
    weights = (avg[pop_col] - frac).apply(lambda x: 1 / np.abs(x))
    avg = avg.mul(weights, axis=0)\
            .sum().reset_index()\
            .rename({'index': 'var', 0: 'frac'}, axis=1)
    avg = avg[avg['var'] != pop_col]
    avg['frac'] /= weights.sum()
    avg['var'] = avg['var'].apply(lambda x: '_'.join(x.split('_')[1:-1]))
    avg['sort'] = sort
    avg['under_over'] = avg.apply(get_under_over, axis=1)
    if demographic:
        avg['demographic'] = demographic
    return avg

bc_df = []
gender_names = ['male', 'female']
age_names = ['lt_30', '30_45', '45_60', 'gte_60']
relig_names = ['christian', 'islam', 'traditional']
lang_names = ['english', 'hausa', 'yoruba', 'igbo', 'other']

for sort in ['nlss', 'dhs', 'didl']:
    bc_df.append(get_barchart_data(gender_df, gender_names, sort, 'Gender'))
    bc_df.append(get_barchart_data(age_df, age_names, sort, 'Age'))
    bc_df.append(get_barchart_data(relig_df, relig_names, sort, 'Religion'))
    bc_df.append(get_barchart_data(lang_df, lang_names, sort, 'Language'))

bc_df = pd.concat(bc_df)

x_df = pd.DataFrame()
x_df['var'] = gender_names + age_names + relig_names + lang_names
x_df['x'] = [i for i in range(len(x_df))]
bc_df = bc_df.merge(x_df)


In [214]:
#@title Bootstrap helper
# choose a sample of wards
# what fraction of poor people in wards are in category?
# what fraction of targeted people in wards are in category?

def get_bootstrapped_ci(col, names, lambdas, hh_data=nlss_data,
                    region='ward', b=1000):
    np.random.seed(123)
    t0 = time.time()

    bs = []
    for i in range(b):
        df = get_parity_data(col, names, lambdas, hh_data, region, sample=True)
        for sort in ['nlss', 'dhs', 'didl']:
            bs.append(get_barchart_data(df, names, sort, 'Gender')) # need to vary sort

    bs = pd.concat(bs)
    bounds = []

    for var in names:
        for sort in ['nlss', 'dhs', 'didl']:
            check = bs.loc[(bs['var'] == var) & (bs['sort'] == sort), 'under_over']
            bounds.append([var, sort, check.quantile(0.025), check.quantile(0.975)])

    bounds = pd.DataFrame(bounds, columns=['var', 'sort', 'upper', 'lower'])
    print(names, 'complete', round(time.time() - t0), 'elapsed')
    return bounds, bs




In [215]:
#@title Bootstrap wards
gender_cis, gender_bs = get_bootstrapped_ci(gender_col, gender_names, gender_lambdas)
age_cis, age_bs = get_bootstrapped_ci(age_col, age_names, age_lambdas)
relig_cis, relig_bs = get_bootstrapped_ci(relig_col, relig_names, relig_lambdas, nlss_aug)
lang_cis, lang_bs = get_bootstrapped_ci(lang_col, lang_names, lang_lambdas,
                               nlss_aug[~nlss_aug['language'].isna()])
cis = pd.concat([gender_cis, age_cis, relig_cis, lang_cis])

gender_bs.to_csv(OUT_DIR + 'gender_bs.csv', index=False)
age_bs.to_csv(OUT_DIR + 'age_bs.csv', index=False)
relig_bs.to_csv(OUT_DIR + 'relig_bs.csv', index=False)
lang_bs.to_csv(OUT_DIR + 'lang_bs.csv', index=False)
cis.to_csv(OUT_DIR + 'cis.csv', index=False)


['male', 'female'] complete 123 elapsed
['lt_30', '30_45', '45_60', 'gte_60'] complete 157 elapsed
['christian', 'islam', 'traditional'] complete 102 elapsed
['english', 'hausa', 'yoruba', 'igbo', 'other'] complete 122 elapsed


In [ ]:
#@title Bootstrap LGAs
# gender_cis_lga = get_bootstrapped_ci(gender_col, gender_names, gender_lambdas, region='lga')
# age_cis_lga = get_bootstrapped_ci(age_col, age_names, age_lambdas, region='lga')
# relig_cis_lga = get_bootstrapped_ci(relig_col, relig_names, relig_lambdas, nlss_aug, region='lga')
# lang_cis_lga = get_bootstrapped_ci(lang_col, lang_names, lang_lambdas,
#                                nlss_aug[~nlss_aug['language'].isna()], region='lga')

# cis_lga = pd.concat([gender_cis_lga, age_cis_lga, relig_cis_lga, lang_cis_lga])
# cis_lga.to_csv(OUT_DIR + 'cis_lga.csv', index=False)

gender_cis, gender_bs = get_bootstrapped_ci(gender_col, gender_names, gender_lambdas, region='lga')
age_cis, age_bs = get_bootstrapped_ci(age_col, age_names, age_lambdas, region='lga')
relig_cis, relig_bs = get_bootstrapped_ci(relig_col, relig_names, relig_lambdas, nlss_aug,
                                          region='lga')
lang_cis, lang_bs = get_bootstrapped_ci(lang_col, lang_names, lang_lambdas,
                                        nlss_aug[~nlss_aug['language'].isna()], region='lga')
cis = pd.concat([gender_cis, age_cis, relig_cis, lang_cis])

gender_bs.to_csv(OUT_DIR + 'gender_bs_lga.csv', index=False)
age_bs.to_csv(OUT_DIR + 'age_bs_lga.csv', index=False)
relig_bs.to_csv(OUT_DIR + 'relig_bs_lga.csv', index=False)
lang_bs.to_csv(OUT_DIR + 'lang_bs_lga.csv', index=False)
cis.to_csv(OUT_DIR + 'cis_lga.csv', index=False)


In [ ]:
#@title Export data to R
# gender_df.to_csv(OUT_DIR + 'gender_parity.csv', index=False)
# age_df.to_csv(OUT_DIR + 'age_parity.csv', index=False)
# relig_df.to_csv(OUT_DIR + 'religion_parity.csv', index=False)
# lang_df.to_csv(OUT_DIR + 'language_parity.csv', index=False)
# bc_df.to_csv(OUT_DIR + 'bar_chart_parity.csv', index=False)

# gender_df_lga.to_csv(OUT_DIR + 'gender_parity_lga.csv', index=False)
# age_df_lga.to_csv(OUT_DIR + 'age_parity_lga.csv', index=False)
# relig_df_lga.to_csv(OUT_DIR + 'religion_parity_lga.csv', index=False)
# lang_df_lga.to_csv(OUT_DIR + 'language_parity_lga.csv', index=False)
# bc_df.to_csv(OUT_DIR + 'bar_chart_parity_lga.csv', index=False)


In [ ]:
#@title R packages
# %load_ext rpy2.ipython
%%R
install.packages('cowplot')
install.packages('gridExtra')
install.packages('Metrics')
install.packages('showtext')


In [ ]:
#@title Setup for R plots
%%R

library(cowplot)
# library(fixest)
library(foreign)
library(grid)
library(gridExtra)
library(Metrics)
library(RColorBrewer)
library(showtext)
library(stringr)

library(ggplot2)
library(dplyr)
library(tidyr)

setwd('/content//drive/MyDrive/poverty_map_paper/')
font_add("LM Roman 10", "fonts/Latin-Modern-Roman/lmroman10-regular.otf")
font_add("LM Roman 10 Bold", "fonts/Latin-Modern-Roman/lmroman10-bold.otf")
showtext.auto()

DATA_DIR <- 'output/figure_data/0721/'

pal <- brewer.pal(n=4, name='RdYlBu')
pal3 <- pal[c(1, 2, 4)]
target_type_pal <- c('black', 'palegreen4', 'lightblue')
gray <- 'gray60'

scatterplot_ptsz <- 1
scatterplot_lnwt <- 0.5
scatterplot_lncol <- pal[4]

scatterplot_theme <- theme_classic() +
  theme(text=element_text(size=28, family="LM Roman 10"),
        plot.title = element_text(hjust = 0.5, size=32),
        legend.position = c(0.65, 0.2),
        legend.title = element_blank(),
        legend.text = element_text(size=14),
        legend.margin = margin(c(0, 0, 0, 0)))


In [ ]:
#@title Parity plot helper
%%R
get_parity_plot <- function(df, group) {
    group_col <- paste('frac_', group, sep='')
    overall_col <- paste(group, '_poor', sep='')
    if (grepl("^[[:digit:]]+", overall_col)) {
        overall_col <- paste('X', overall_col, sep='')
    }
    overall <- df[1, overall_col]

    plt<- ggplot(data=df) +
        geom_line(aes_string(x='frac_pop_nlss', y=paste(group_col, '_nlss', sep='')),
                  size=scatterplot_lnwt, color=target_type_pal[1]) +
        geom_line(aes_string(x='frac_pop_didl', y=paste(group_col, '_didl', sep='')),
                  size=scatterplot_lnwt, color=target_type_pal[2]) +
        geom_line(aes_string(x='frac_pop_dhs', y=paste(group_col, '_dhs', sep='')),
                  size=scatterplot_lnwt, color=target_type_pal[3]) +
        geom_abline(intercept=overall, slope=0, linetype='dashed', color=gray,
                    size=scatterplot_lnwt) +
        ylim(0, 1) +
        labs(x='Fraction of Population Targeted',
             y=paste('Fraction of Targeted Households', str_to_title(group))) +
        scatterplot_theme
    return(plt)
}

In [ ]:
#@title Get R plots
%%R
w = 4

gender <- read.csv('output/figure_data/0721/gender_parity.csv')
plt_f <- get_parity_plot(gender, 'female')
plt_m <- get_parity_plot(gender, 'male')
plt_gender <- plot_grid(plt_f, plt_m, nrow=1)
ggsave("output/figures/gender.png", units="in", width=w*2, height=w, dpi=300)

age <- read.csv('output/figure_data/0721/age_parity.csv')
plt_lt_30 <- get_parity_plot(age, 'lt_30')
plt_30_45 <- get_parity_plot(age, '30_45')
plt_45_50 <- get_parity_plot(age, '45_60')
plt_gte_60 <- get_parity_plot(age, 'gte_60')
plt_age <- plot_grid(plt_lt_30, plt_30_45, plt_45_50, plt_gte_60, nrow=2)
ggsave("output/figures/age.png", units="in", width=w*2, height=w*2, dpi=300)

religion <- read.csv('output/figure_data/0721/religion_parity.csv')
head(religion)
plt_christian <- get_parity_plot(religion, 'christian');
plt_islam <- get_parity_plot(religion, 'islam');
plt_traditional <- get_parity_plot(religion, 'traditional');
plt_religion <- plot_grid(plt_christian, plt_islam, plt_traditional,
                          nrow=2)
ggsave("output/figures/religion.png", units="in", width=w*2, height=w*2, dpi=300)

lang <- read.csv('output/figure_data/0721/language_parity.csv')

plt_english <- get_parity_plot(lang, 'english');
plt_hausa <- get_parity_plot(lang, 'hausa');
plt_igbo <- get_parity_plot(lang, 'igbo');
plt_yoruba <- get_parity_plot(lang, 'yoruba');
plt_other <- get_parity_plot(lang, 'other');
plt_lang <- plot_grid(plt_english, plt_hausa, plt_igbo, plt_yoruba, plt_other)#
                    #   nrow=2)
ggsave("output/figures/lang.png", units="in", width=w*2, height=w*2, dpi=300)




In [ ]:
#@title Test plots
%%R

bc_data <- read.csv('output/figure_data/0721/bar_chart_parity.csv')

bc <- ggplot(data=bc_data) +
    geom_point(aes(x=x, y=under_over, color=sort), size=3) +
    labs(x=bc_data['var']) +
    scale_color_manual(values=c('black', 'palegreen4', 'lightblue'))#, labels=labels)
    # scatterplot_theme
    

show(bc)

# # Python
# import imageio
# im = imageio.imread('output/figures/lang.png')

# f, ax = plt.subplots(figsize=(12, 12))
# ax.imshow(im)

In [ ]:
#@title Test plots
f, ax = plt.subplots()

sns.lineplot(x=gender_plot_df['frac_pop_nlss'],
             y=gender_plot_df['frac_f_nlss'],
             color='red')
sns.lineplot(x=gender_plot_df['frac_pop_dhs'],
             y=gender_plot_df['frac_f_dhs'],
             color='green')
sns.lineplot(x=gender_plot_df['frac_pop_didl'],
             y=gender_plot_df['frac_f_didl'],
             color='blue')
sns.lineplot(x=[0, 1],
             y=[frac_of_poor_female, frac_of_poor_female])


f, ax = plt.subplots()

sns.lineplot(x=gender_plot_df['frac_pop_nlss'],
             y=gender_plot_df['frac_m_nlss'],
             color='red')
sns.lineplot(x=gender_plot_df['frac_pop_dhs'],
             y=gender_plot_df['frac_m_dhs'],
             color='green')
sns.lineplot(x=gender_plot_df['frac_pop_didl'],
             y=gender_plot_df['frac_m_didl'],
             color='blue')
sns.lineplot(x=[0, 1],
             y=[frac_of_poor_male, frac_of_poor_male])



# Make table 1

In [217]:
%load_ext rpy2.ipython
%R install.packages('psychometric')


R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/tmp/Rtmph6itBe/downloaded_packages’
R[write to console]: 
R[write to console]: 



In [218]:
#@title Table 1 helpers
from rpy2.robjects.packages import importr
psychometric = importr('psychometric')

def corr(df, c1, c2):
    r, p = pearsonr(df[c1], df[c2])
    # if p < 0.0001:
    #     return '%.3f (p<0.0001)' % r
    # return '%.3f (p=%.4f)' % (r, p)
    ci = psychometric.CIr(r=float(r), n=len(df), level = .95)
    return '%.3f [%.3f, %.3f] (p=%.2e)' % (r, ci[0], ci[1], p)


def print_row(dhs_df, ml_df):
    print('n DHS: %d, n ML: %d, corr DHS: %s, corr ML: %s' %
      (len(dhs_df),
       len(ml_df),
       corr(dhs_df, 'nlss_rwi', 'dhs_rwi'),
       corr(ml_df, 'nlss_rwi', 'didl_orig_rwi')))

def print_ml_only_row(ml_df):
    print('n ML: %d, corr ML: %s' % (len(ml_df), corr(ml_df, 'nlss_rwi', 'didl_orig_rwi')))


In [219]:
#@title Get table 1 dfs
# State data
state_data_table_1 = nlss_state[['state', 'nlss_rwi']].merge(
                     didl_orig_state[['state', 'didl_orig_rwi']]).merge(
                     dhs_state[['state', 'dhs_rwi']])

# LGA data
lga_clusters = nlss_pls.groupby(['lga'])['hhid'].nunique().reset_index()\
    .merge(nlss_lga)[['lga', 'hhid', 'nlss_rwi']]

lgas_nlss_ml = lga_clusters.merge(didl_orig_lga[['lga', 'didl_orig_rwi']])
lgas_all_three = lgas_nlss_ml.merge(dhs_lga[['lga', 'dhs_rwi']])
lgas_no_dhs = lgas_nlss_ml[~lgas_nlss_ml['lga'].isin(lgas_all_three['lga'])]

# Ward data
ward_clusters = nlss_pls.groupby(['ward'])['hhid'].nunique().reset_index()\
    .merge(nlss_ward)[['ward', 'hhid', 'nlss_rwi']]

wards_nlss_ml = ward_clusters.merge(didl_orig_ward[['ward', 'didl_orig_rwi']])
wards_all_three = wards_nlss_ml.merge(dhs_ward[['ward', 'dhs_rwi']])
wards_no_dhs = wards_nlss_ml[~wards_nlss_ml['ward'].isin(wards_all_three['ward'])]


In [ ]:
#@title Print table 1 rows
# Table 1 state rows
print('All states:')
print_row(state_data_table_1, state_data_table_1)
print()

# Table 1 LGA rows
print('All LGAs')
print_row(lgas_all_three, lgas_nlss_ml)
print('LGAs with DHS data')
print_ml_only_row(lgas_all_three)
print('>30 ground truth households')
print_row(lgas_all_three[lgas_all_three['hhid'] >= 30],
          lgas_nlss_ml[lgas_nlss_ml['hhid'] >= 30])
print('>30 ground truth households and DHS data')
print_ml_only_row(lgas_all_three[lgas_all_three['hhid'] >= 30])
print('LGAs with no DHS data')
print_ml_only_row(lgas_no_dhs)
print('>30 ground truth households and no DHS')
print_ml_only_row(lgas_no_dhs[lgas_no_dhs['hhid'] >= 30])
print()

# Table 1 ward rows
print('All wards')
print_row(wards_all_three, wards_nlss_ml)
print('Wards with DHS data')
print_ml_only_row(wards_all_three)
print('>20 ground truth households')
print_row(wards_all_three[wards_all_three['hhid'] >= 20],
          wards_nlss_ml[wards_nlss_ml['hhid'] >= 20])
print('>20 ground truth households and DHS data')
print_ml_only_row(wards_all_three[wards_all_three['hhid'] >= 20])
print('Wards with no DHS data')
print_ml_only_row(wards_no_dhs)
print('>20 ground truth households and no DHS')
print_ml_only_row(wards_no_dhs[wards_no_dhs['hhid'] >= 20])
print()



# Misc - tables and appendix

##Make cluster locations fig

In [ ]:
#@title Get NLSS df with displaced coordinates
# DHS displacement methodology: https://dhsprogram.com/pubs/pdf/SAR8/SAR8.pdf (p. 17)
np.random.seed(123)
nlss_plot = nlss_data[['cluster', 'geometry']]
nlss_plot['longitude'] = nlss_data['geometry'].x
nlss_plot['latitude'] = nlss_data['geometry'].y
nlss_plot = nlss_plot.groupby('cluster')[['longitude', 'latitude']].mean().reset_index()\
    .merge(nlss_data[['cluster', 'sector']].drop_duplicates(subset='cluster'))
nlss_plot['disp_radius'] = \
    nlss_plot['sector'].apply(lambda x: 10 * 0.008 if np.random.random() <= 1/100 else
                              2 * 0.008 if x == '1. Urban' else 5 * 0.008)

def get_random(x):
    alpha = 2 * math.pi * np.random.random()
    r = x['disp_radius'] * math.sqrt(random.random())
    return Point(r * math.cos(alpha) + x['longitude'],
                 r * math.sin(alpha) + x['latitude'])

nlss_plot['geometry'] = nlss_plot.apply(get_random, axis=1)


In [ ]:
#@title Make cluster locations fig
import matplotlib.font_manager as fm
bold_font = fm.FontProperties(
    fname='drive/MyDrive/poverty_map_paper/Latin-Modern-Roman/lmroman10-bold.otf')
cmap = matplotlib.cm.get_cmap('YlGnBu')

dhs_plot = dhs_eval[~dhs_eval['ward'].isna()].drop_duplicates(subset='cluster_id')

sns.set_style('white')
f, ax = plt.subplots(1, 2, figsize=(14, 8))
plt.subplots_adjust(wspace=0, hspace=0)
country.plot(ax=ax[0], edgecolor='darkgray', color='none')
gpd.GeoDataFrame(nlss_plot).plot(ax=ax[0], markersize=1, color=cmap(0.35))

country.plot(ax=ax[1], edgecolor='darkgray', color='none')
dhs_plot.plot(ax=ax[1], markersize=1, color=cmap(0.8))

ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].set_title('A. NLSS Cluster Locations', fontproperties=bold_font, size=18)
ax[1].set_title('B. DHS Cluster Locations', fontproperties=bold_font, size=18)

plt.savefig(FIG_PATH + 'cluster_locs.png', dpi=300, bbox_inches='tight', pad_inches=0.2)


##Make table 2

In [ ]:
#@title Table 2 helpers
def get_precision(pl_df, num_poor_col, recall=0.10):
    n_poor = pl_df[num_poor_col].sum() * recall
    included = pl_df[pl_df[num_poor_col].cumsum() <= n_poor]
    included_poor = included[num_poor_col].sum()

    partial = pl_df[pl_df[num_poor_col].cumsum() >= n_poor].iloc[0]
    included_pop = included['pop'].sum() + \
        (n_poor - included_poor) / partial[num_poor_col] * partial['pop']
    return n_poor / included_pop

def get_table2_row(pl_df, ultrapoor_df=None):
    ultrapoor_df = ultrapoor_df if ultrapoor_df is not None else pl_df
    prec_poor = get_precision(pl_df, 'asset_poor_w')
    prec_ultrapoor = get_precision(ultrapoor_df, 'asset_ultrapoor_w')
    return '%.3f, %.3f' % (prec_poor, prec_ultrapoor)

In [ ]:
#@title Get table 2 (main specification)
print('States')
print('NLSS', get_table2_row(nlss_pls_state.sort_values('nlss_rwi')))
print('ML', get_table2_row(didl_pls_state.sort_values('didl_orig_rwi')))
print('DHS', get_table2_row(dhs_pls_state.sort_values('dhs_rwi')))

print('LGAs')
print('NLSS', get_table2_row(nlss_pls_lga.sort_values('nlss_rwi')))
print('ML', get_table2_row(didl_pls_lga.sort_values('didl_orig_rwi')))
print('DHS', get_table2_row(noisy_dhs_lga_poor.sort_values('rwi')))
print('DHS upper bound', get_table2_row(dhs_pls_lga.sort_values('dhs_rwi')))

print('Wards')
print('NLSS', get_table2_row(nlss_pls_ward.sort_values('nlss_rwi')))
print('ML', get_table2_row(didl_pls_ward.sort_values('didl_orig_rwi')))
print('DHS', get_table2_row(noisy_dhs_ward_poor.sort_values('rwi')))
print('DHS upper bound', get_table2_row(dhs_pls_ward.sort_values('dhs_rwi')))

In [ ]:
#@title Get accuracy figure data
def get_acc(pl_df, num_poor_col, frac_targeted):
    n_targeted = nlss_pls_ward['asset_poor_w'].sum() * frac_targeted
    included = pl_df[pl_df['cum_pop'] <= n_targeted]
    included_poor = included[num_poor_col].sum()

    partial = pl_df[pl_df['cum_pop'] >= n_targeted].iloc[0]
    partial_frac = (n_targeted - included['pop'].sum()) / partial['pop']
    included_poor += partial_frac * partial[num_poor_col]

    return included_poor / n_targeted, len(included)

df = {'acc_didl': [], 'n_didl': [], 'acc_dhs': [], 'n_dhs': [], 'thresh': []}
for i in range(1, 101):
    acc_didl, n_didl = \
        get_acc(didl_pls_ward.sort_values('didl_orig_rwi'), 'asset_ultrapoor_w', i / 100)
    acc_dhs, n_dhs = \
        get_acc(noisy_dhs_ward_ultrapoor.sort_values('rwi'), 'asset_ultrapoor_w', i / 100)

    df['acc_didl'].append(acc_didl)
    df['n_didl'].append(n_didl)
    df['acc_dhs'].append(acc_dhs)
    df['n_dhs'].append(n_dhs)
    df['thresh'].append(i / 100)

df = pd.DataFrame(df)



In [ ]:
overlap = df[df['acc_didl'] < df['acc_dhs']]
didl_start, didl_end = overlap['n_didl'].min(), overlap['n_didl'].max()
dhs_start, dhs_end = overlap['n_dhs'].min(), overlap['n_dhs'].max()

didl_wards = didl_pls_ward.sort_values('didl_orig_rwi')[didl_start - 1: didl_end]
didl_wards = didl_wards[['ward', 'didl_orig_rwi']]\
    .merge(nlss_pls_ward[['ward', 'asset_ultrapoor', 'nlss_rwi', 'pop']])

dhs_wards = noisy_dhs_ward_ultrapoor.sort_values('rwi')[dhs_start - 1: dhs_end]
dhs_wards = dhs_wards[['ward', 'rwi']]\
    .merge(nlss_pls_ward[['ward', 'asset_ultrapoor', 'nlss_rwi', 'pop']])

didl_wards['nlss_rwi'].mean(), dhs_wards['nlss_rwi'].mean(), \
didl_wards['asset_ultrapoor'].mean(), dhs_wards['asset_ultrapoor'].mean()



##Results for all NLSS LGAs/wards

In [ ]:
#@title Get PL dfs for full NLSS sample of LGAs and wards
# Get data for all NLSS LGAs
dhs_imputed_lgas = lgas.merge(dhs_lga, how='left')\
    .merge(dhs_state.rename({'dhs_rwi': 'state_rwi'}, axis=1), how='left')\
    .merge(nlss_lga)
dhs_imputed_lgas['dhs_rwi'] = dhs_imputed_lgas['dhs_rwi'].fillna(dhs_imputed_lgas['state_rwi'])

nlss_pls_lga_all = get_region_pl_df_uw(lgas, 'lga')
dhs_pls_lga_all = get_dhs_pl_df('lga', dhs_imputed_lgas, nlss_pls_lga_all)
didl_pls_lga_all = get_didl_pl_df('lga', didl_orig_lga, nlss_pls_lga_all)
# get_roc(dhs_pls_lga_all)
# get_roc(didl_pls_lga_all)

# Get LGA data for all NLSS wards
dhs_imputed_wards = wards.merge(dhs_ward, how='left')\
    .merge(dhs_lga.rename({'dhs_rwi': 'lga_rwi'}, axis=1), how='left')\
    .merge(dhs_state.rename({'dhs_rwi': 'state_rwi'}, axis=1), how='left')\
    .merge(nlss_ward)
dhs_imputed_wards['dhs_rwi'] = dhs_imputed_wards['dhs_rwi'].fillna(dhs_imputed_wards['lga_rwi'])\
                                                           .fillna(dhs_imputed_wards['state_rwi'])

nlss_pls_ward_all = get_region_pl_df_uw(wards, 'ward')
dhs_pls_ward_all = get_dhs_pl_df('ward', dhs_imputed_wards, nlss_pls_ward_all)
didl_pls_ward_all = get_didl_pl_df('ward', didl_orig_ward, nlss_pls_ward_all)
# get_roc(dhs_pls_ward_all)
# get_roc(didl_pls_ward_all)

print('Fraction of NLSS wards with DHS data: %.1f' % (len(roc_wards) / len(nlss_ward) * 100))
print('Fraction of all wards with DHS data: %.1f' % (len(dhs_ward) / len(wards) * 100))

print('Fraction of NLSS LGAs with DHS data: %.1f' % (len(roc_lgas) / len(nlss_lga) * 100))
print('Fraction of all LGAs with DHS data: %.1f' % (len(dhs_lga) / len(lgas) * 100))


706
2016
Fraction of NLSS wards with DHS data: 23.0
Fraction of all wards with DHS data: 13.8
Fraction of NLSS LGAs with DHS data: 84.6
Fraction of all LGAs with DHS data: 81.5


In [ ]:
#@title Get table 2, imputing NLSS regions
print('LGAs')
print('NLSS', get_table2_row(nlss_pls_lga_all.sort_values('nlss_rwi')))
print('ML', get_table2_row(didl_pls_lga_all.sort_values('didl_orig_rwi')))
print('DHS', get_table2_row(dhs_pls_lga_all.sort_values('dhs_rwi')))

print('Wards')
print('NLSS', get_table2_row(nlss_pls_ward_all.sort_values('nlss_rwi')))
print('ML', get_table2_row(didl_pls_ward_all.sort_values('didl_orig_rwi')))
print('DHS', get_table2_row(dhs_pls_ward_all.sort_values('dhs_rwi')))


In [ ]:
#@title Get ROC
print('NLSS')
get_roc(nlss_pls_ward_all)
print('Satellite')
get_roc(didl_pls_ward_all)
print('DHS')
get_roc(dhs_pls_ward_all)


In [ ]:
#@title Get AUC data for R
keep_cols = ['asset_nonpoor_inc', 'asset_poor_inc',
             'asset_ultrapoor_inc', 'asset_nonultrapoor_inc']
r_ward_nlss = nlss_pls_ward_all[keep_cols]
r_ward_didl = didl_pls_ward_all[keep_cols]
r_ward_dhs = dhs_pls_ward_all[keep_cols]

r_ward_nlss['type'] = 'Optimal'
r_ward_didl['type'] = 'Satellite'
r_ward_dhs['type'] = 'DHS'

r_ward_roc_df = pd.concat([r_ward_nlss, r_ward_didl, r_ward_dhs])
r_ward_roc_df.to_csv(OUT_DIR + 'roc_plot_data_all_wards.csv', index=False)
r_ward_roc_df



In [ ]:
#@title Get accuracy data for R
targeting_approach, poor, ultrapoor, thresh = [], [], [], []
nlss_pls_ward_all = nlss_pls_ward_all.sort_values('nlss_rwi')
didl_pls_ward_all = didl_pls_ward_all.sort_values('didl_orig_rwi')
dhs_pls_ward_all = dhs_pls_ward_all.sort_values('dhs_rwi')

for i in range(1, 100):
    thresh += [i / 100] * 3
    targeting_approach += ['Optimal', 'Satellite', 'DHS']
    poor += [get_precision(nlss_pls_ward_all, 'asset_poor_w', recall=i/100),
             get_precision(didl_pls_ward_all, 'asset_poor_w', recall=i/100),
             get_precision(dhs_pls_ward_all, 'asset_poor_w', recall=i/100)]
    ultrapoor += [get_precision(nlss_pls_ward_all, 'asset_ultrapoor_w', recall=i/100),
                  get_precision(didl_pls_ward_all, 'asset_ultrapoor_w', recall=i/100),
                  get_precision(dhs_pls_ward_all, 'asset_ultrapoor_w', recall=i/100)]

acc_df = pd.DataFrame({'targeting_approach': targeting_approach, 'poor': poor,
                       'verypoor': ultrapoor, 'thresh': thresh})
acc_df.to_csv(OUT_DIR + 'acc_plot_data_all_wards.csv', index=False)
    

# Target with DHS as gt and NLSS as survey benchmark

In [ ]:
#@title Make DHS poverty line df

print(poor_thresh, ultrapoor_thresh)

dhs_pls = dhs_eval[['weight', 'dhs_rwi', 'ward', 'lga', 'geo_state']]\
    .rename({'geo_state': 'state'}, axis=1)
dhs_pls['popw'] = dhs_pls['weight'] * (nlss_pls['popw'].sum() / dhs_pls['weight'].sum())
dhs_pls['dhs_rwi_w'] = dhs_pls['dhs_rwi'] * dhs_pls['popw']

dhs_pls = get_weighted_quantile(dhs_pls, 'dhs_rwi', 'weight', 'dhs_rwi_q')
pop_reweight = states['pop'].sum() / dhs_pls['popw'].sum()

dhs_pls['asset_poor'] = dhs_pls['dhs_rwi_q'].apply(lambda x: 0 if x > poor_thresh else 1)
dhs_pls['asset_ultrapoor'] = dhs_pls['dhs_rwi_q'].apply(lambda x: 0 if x > ultrapoor_thresh else 1)

dhs_pls['asset_poor_w'] = dhs_pls.apply(
    lambda x: x['asset_poor'] * x['popw'] * pop_reweight, axis=1)
dhs_pls['asset_ultrapoor_w'] = dhs_pls.apply(
    lambda x: x['asset_ultrapoor'] * x['popw'] * pop_reweight, axis=1)

total_poor = states['pop'].sum() * poor_thresh
total_ultrapoor = states['pop'].sum() * ultrapoor_thresh
total_nonpoor = states['pop'].sum() * (1 - poor_thresh)


0.40532921993869897 0.08207280524503148


In [ ]:
#@title DHS state level targeting DF
dhs_gt_pls_state = dhs_pls.groupby('state')\
    [['dhs_rwi_w', 'asset_poor_w', 'asset_ultrapoor_w']]\
    .sum().reset_index().merge(states)

dhs_gt_pls_state['dhs_rwi'] = dhs_gt_pls_state['dhs_rwi_w'] \
    / dhs_gt_pls_state['pop']
dhs_gt_pls_state = dhs_gt_pls_state.sort_values(by='dhs_rwi')\
                                   .drop('dhs_rwi_w', axis=1)
dhs_gt_pls_state['cum_pop'] = dhs_gt_pls_state['pop'].cumsum()

dhs_gt_pls_state['asset_poor_inc'] = dhs_gt_pls_state['asset_poor_w'].cumsum() \
    / total_poor
dhs_gt_pls_state['asset_ultrapoor_inc'] = dhs_gt_pls_state['asset_ultrapoor_w']\
    .cumsum() / total_ultrapoor
dhs_gt_pls_state['asset_nonpoor_inc'] = (dhs_gt_pls_state['cum_pop'] -
    dhs_gt_pls_state['asset_poor_w'].cumsum()) \
    / (states['pop'].sum() * (1 - poor_thresh))
dhs_gt_pls_state['asset_nonultrapoor_inc'] = (dhs_gt_pls_state['cum_pop'] -
    dhs_gt_pls_state['asset_ultrapoor_w'].cumsum()) \
    / (states['pop'].sum() * (1 - ultrapoor_thresh))


In [ ]:
#@title DHS LGA and ward level targeting DFs
# DHS poverty lines, LGA
dhs_gt_pls_lga = get_region_pl_df_uw(roc_lgas, 'lga', 'dhs_rwi_w', 'dhs_rwi', dhs_pls)
print('Asset poor frac, lgas:', round(dhs_gt_pls_lga['asset_poor_w'].sum() /
                                      dhs_gt_pls_lga['pop'].sum(), 3))
print('Asset very poor frac, lgas:', round(dhs_gt_pls_lga['asset_ultrapoor_w'].sum() /
                                           dhs_gt_pls_lga['pop'].sum(), 4))

# DHS poverty lines, ward
dhs_gt_pls_ward = get_region_pl_df_uw(roc_wards, 'ward', 'dhs_rwi_w', 'dhs_rwi', dhs_pls)
print('Asset poor frac, wards:', round(dhs_gt_pls_ward['asset_poor_w'].sum() /
                                       dhs_gt_pls_ward['pop'].sum(), 3))
print('Asset very poor frac, wards:', round(dhs_gt_pls_ward['asset_ultrapoor_w'].sum() /
                                            dhs_gt_pls_ward['pop'].sum(), 4))


597
Asset poor frac, lgas: 0.4
Asset very poor frac, lgas: 0.0819
464
Asset poor frac, wards: 0.271
Asset very poor frac, wards: 0.0541


In [ ]:
# @title Get DIDL data for ROC plot, DHS ground truth
didl_pls_state_dhsgt = get_didl_pl_df('state', didl_orig_state, dhs_gt_pls_state)
didl_pls_lga_dhsgt = get_didl_pl_df('lga', didl_orig_lga, dhs_gt_pls_lga)
didl_pls_ward_dhsgt = get_didl_pl_df('ward', didl_orig_ward, dhs_gt_pls_ward)


(597, 597)

In [ ]:
#@title Get NLSS data for ROC plot, DHS ground truth

def get_pl_df(groupby_col, eval_reg, gt_reg, eval_rwi_col):
    eval_pls_reg = gt_reg.merge(eval_reg[[groupby_col, eval_rwi_col]])\
                              .sort_values(by=eval_rwi_col)
    return get_inc_ex(eval_pls_reg)

nlss_pls_state_dhsgt = get_pl_df('state', nlss_pls_state, dhs_gt_pls_state, 'nlss_rwi')
nlss_pls_lga_dhsgt = get_pl_df('lga', nlss_pls_lga, dhs_gt_pls_lga, 'nlss_rwi')
nlss_pls_ward_dhsgt = get_pl_df('ward', nlss_pls_ward, dhs_gt_pls_ward, 'nlss_rwi')


In [ ]:
#@title Make imputed NLSS df -- LGAs, DHS ground truth
def impute_lga_with_state_dhsgt(row):
    drop_lga_df = nlss_hhs_dhsgt.loc[(nlss_hhs_dhsgt['lga'] != row['lga']) &
                                     (nlss_hhs_dhsgt['state'] == row['state'])]
    return drop_lga_df['nlss_rwi_w'].sum() / drop_lga_df['popw'].sum()

nlss_hhs_dhsgt = nlss_hhs.copy()
nlss_hhs_dhsgt['nlss_rwi_w'] = nlss_hhs_dhsgt['popw'] * standardize(nlss_hhs_dhsgt['nlss_rwi'])

nlss_lga_impute = nlss_lga.merge(lgas[['lga', 'state']]) #, how='right')
nlss_lga_impute['imputed_rwi'] = nlss_lga_impute.apply(impute_lga_with_state_dhsgt, axis=1)
print(round(pearsonr(nlss_lga_impute['imputed_rwi'], nlss_lga_impute['nlss_rwi'])[0], 3))


0.656


In [ ]:
#@title Make imputed NLSS df -- wards, DHS ground truth
def impute_ward_with_lga_dhsgt(row):
    drop_ward_df = nlss_hhs_dhsgt.loc[(nlss_hhs_dhsgt['ward'] != row['ward']) &
                                      (nlss_hhs_dhsgt['lga'] == row['lga'])]
    if len(drop_ward_df) == 0:
        return None
    return drop_ward_df['nlss_rwi_w'].sum() / drop_ward_df['popw'].count()

nlss_hhs_dhsgt['nlss_rwi'] = standardize(nlss_hhs_dhsgt['nlss_rwi'])
nlss_ward_impute = nlss_ward.merge(wards[['ward', 'lga']]) #, how='right')

nlss_ward_impute['imputed_rwi'] = nlss_ward_impute.apply(impute_ward_with_lga_dhsgt, axis=1)

nlss_ward_impute = nlss_ward_impute.merge(nlss_lga_impute[['lga', 'imputed_rwi']].
                                          rename({'imputed_rwi': 'state_rwi'}, axis=1),
                                          on='lga')
print(len(nlss_ward_impute))
nlss_ward_impute.loc[nlss_ward_impute['imputed_rwi'].isna(), 'imputed_rwi'] = \
    nlss_ward_impute.loc[nlss_ward_impute['imputed_rwi'].isna(), 'state_rwi']

print(round(pearsonr(nlss_ward_impute['imputed_rwi'], nlss_ward_impute['nlss_rwi'])[0], 3))


2016
0.566


In [ ]:
#@title Bootstrap LGAs w imputation, NLSS - DHS ground truth
np.random.seed(123)

n_to_replace = round((1 - (len(nlss_lga) / len(lgas))) * len(nlss_pls_lga))
nlss_lga_impute = nlss_lga_impute[nlss_lga_impute['lga'].isin(roc_lgas['lga'])]
print(n_to_replace, len(nlss_lga_impute))

lga_output = []
b = 1000
for i in range(b):
    lga_pls_i = get_random_roc('lga', lgas, n_to_replace, nlss_lga_impute,
                               dhs_gt_pls_lga, 'nlss_rwi')
    lga_output += [(get_auc(lga_pls_i, 'asset_nonpoor_inc', 'asset_poor_inc'),
                    get_auc(lga_pls_i, 'asset_nonultrapoor_inc', 'asset_ultrapoor_inc'),
                    lga_pls_i)]

noisy_nlss_lga_poor = sorted(lga_output, key=lambda x: x[0])[int(b / 2) - 1][2]
noisy_nlss_lga_ultrapoor = sorted(lga_output, key=lambda x: x[1])[int(b / 2) - 1][2]


52 597


In [ ]:
#@title Bootstrap wards w imputation, NLSS - DHS ground truth
np.random.seed(123)

n_to_replace = round((1 - (len(nlss_ward) / len(wards))) * len(nlss_pls_ward))
nlss_ward_impute = nlss_ward_impute[nlss_ward_impute['ward'].isin(roc_wards['ward'])]
print(n_to_replace, len(nlss_ward_impute))

ward_output = []
for i in range(b):
    ward_pls_i = get_random_roc('ward', wards, n_to_replace, nlss_ward_impute,
                                dhs_gt_pls_ward, 'nlss_rwi')
    ward_output += [(get_auc(ward_pls_i, 'asset_nonpoor_inc', 'asset_poor_inc'),
                     get_auc(ward_pls_i, 'asset_nonultrapoor_inc', 'asset_ultrapoor_inc'),
                     ward_pls_i)]

noisy_nlss_ward_poor = sorted(ward_output, key=lambda x: x[0])[int(b / 2) - 1][2]
noisy_nlss_ward_ultrapoor = sorted(ward_output, key=lambda x: x[1])[int(b / 2) - 1][2]


358 464


In [ ]:
#@title Get ROC - DHS ground truth
print('DHS')
get_roc(dhs_gt_pls_state)
get_roc(dhs_gt_pls_lga)
get_roc(dhs_gt_pls_ward)
print('\nSatellite')
get_roc(didl_pls_state_dhsgt)
get_roc(didl_pls_lga_dhsgt)
get_roc(didl_pls_ward_dhsgt)
print('\nNLSS')
get_roc(nlss_pls_state_dhsgt)
get_roc(noisy_nlss_lga_poor)
get_roc(noisy_nlss_lga_ultrapoor)
get_roc(noisy_nlss_ward_poor)
get_roc(noisy_nlss_ward_ultrapoor)


DHS
poor 0.812 very poor 0.83
poor 0.918 very poor 0.922
poor 0.955 very poor 0.96

Satellite
poor 0.799 very poor 0.824
poor 0.871 very poor 0.856
poor 0.904 very poor 0.876

NLSS
poor 0.816 very poor 0.852
poor 0.844 very poor 0.832
poor 0.844 very poor 0.834
poor 0.808 very poor 0.816
poor 0.815 very poor 0.817


In [ ]:
#@title Get ward data for R - DHS ground truth
keep_cols = ['asset_nonpoor_inc', 'asset_poor_inc',
             'asset_ultrapoor_inc', 'asset_nonultrapoor_inc']
r_ward_dhs = dhs_gt_pls_ward[keep_cols]
r_ward_didl = didl_pls_ward_dhsgt[keep_cols]
# r_ward_dhs = dhs_pls_ward[keep_cols]
r_noisy_nlss_ward_poor = noisy_nlss_ward_poor[keep_cols]\
    .sort_values('asset_nonpoor_inc')
r_noisy_nlss_ward_ultrapoor = noisy_nlss_ward_ultrapoor[keep_cols]\
    .sort_values('asset_nonultrapoor_inc')

r_ward_dhs['type'] = 'Optimal - DHS'
r_ward_didl['type'] = 'Satellite'
# r_ward_dhs['type'] = 'DHS'
r_noisy_nlss_ward_poor['type'] = 'Survey Benchmark - NLSS'
r_noisy_nlss_ward_ultrapoor['type'] = 'Survey Benchmark - NLSS'

r_ward_roc_df_poor = pd.concat([r_ward_dhs, r_ward_didl, #r_ward_dhs,
                                r_noisy_nlss_ward_poor])
r_ward_roc_df_poor.to_csv(OUT_DIR + 'ward_roc_poor_dhsgt.csv', index=False)
r_ward_roc_df_ultrapoor = pd.concat([r_ward_dhs, r_ward_didl, #r_ward_dhs,
                                     r_noisy_nlss_ward_ultrapoor])
r_ward_roc_df_ultrapoor.to_csv(OUT_DIR + 'ward_roc_ultrapoor_dhsgt.csv', index=False)


# Misc. appendix plots

## Coverage fig

In [ ]:
#@title Get data
didl_orig_all['estimated_rwi'] = standardize(didl_orig_all['estimated_rwi'])
# plot_wards = wards[['ward', 'geometry']].merge(didl_orig_ward)
plot_lgas = lgas[['lga', 'geometry']].merge(didl_orig_lga)
plot_lgas['didl_orig_rwi'] = standardize(plot_lgas['didl_orig_rwi'])
plot_lgas.plot(ax=ax, column='didl_orig_rwi', edgecolor='none', cmap='afmhot')


In [ ]:
#@title Make figure
sns.set_style('white')
f, ax = plt.subplots(3, 2, figsize=(14, 24))
plt.subplots_adjust(wspace=0, hspace=-0.4)

for row in range(3):
    for col in range(2):
        country.plot(ax=ax[row][col], edgecolor='darkgray', color='none')
        for spine in ['top', 'bottom', 'left', 'right']:
            ax[row][col].spines[spine].set_visible(False)
        ax[row][col].xaxis.set_ticks([])
        ax[row][col].yaxis.set_ticks([])

    wards.plot(ax=ax[row][0], edgecolor='darkgray', color='none', linewidth=0.2)
    lgas.plot(ax=ax[row][1], edgecolor='darkgray', color='none', linewidth=0.2)
    
nlss_wards.plot(ax=ax[0][0], edgecolor='darkgray', color='black', linewidth=0.2)
didl_wards.plot(ax=ax[1][0], edgecolor='darkgray', color='#548b54', linewidth=0.2)
dhs_wards.plot(ax=ax[2][0], edgecolor='darkgray', color='lightblue', linewidth=0.2)

nlss_lgas.plot(ax=ax[0][1], edgecolor='darkgray', color='black', linewidth=0.2)
didl_lgas.plot(ax=ax[1][1], edgecolor='darkgray', color='#548b54', linewidth=0.2)
dhs_lgas.plot(ax=ax[2][1], edgecolor='darkgray', color='lightblue', linewidth=0.2)

ax[0][0].set_title('A. Wards Targeted', fontproperties=bold_font, size=18)
ax[0][1].set_title('B. LGAs Targeted', fontproperties=bold_font, size=18)

ax[0][0].set_ylabel('Optimal (NLSS)', fontproperties=bold_font, size=18)
ax[1][0].set_ylabel('ML-Based', fontproperties=bold_font, size=18)
ax[2][0].set_ylabel('Survey Benchmark', fontproperties=bold_font, size=18)
# plt.savefig(FIG_PATH + 'targeted_units_10pct.png', dpi=300, bbox_inches='tight', pad_inches=0.2)


## Appendix figs

In [ ]:
#@title Get regions at 10% of pop targeted, original sample
frac = 0.1
to_target = nlss_pls_lga['pop'].sum() * frac

def get_regions(df, frac, region_col, region_df):
    n = df['pop'].sum() * frac
    full = df.loc[df['cum_pop'] < n, [region_col]].assign(partial=0)
    partial = df.loc[df['cum_pop'] >= n][[region_col]].assign(partial=1)[:1]
    return region_df.merge(full) #region_df.merge(pd.concat([full, partial]))

nlss_lgas = get_regions(nlss_pls_lga.sort_values('nlss_rwi'), 0.1, 'lga', lgas)
dhs_lgas = get_regions(noisy_dhs_lga_poor.sort_values('rwi'), 0.1, 'lga', lgas)
didl_lgas = get_regions(didl_pls_lga.sort_values('didl_orig_rwi'), 0.1, 'lga', lgas)

nlss_wards = get_regions(nlss_pls_ward.sort_values('nlss_rwi'), 0.1, 'ward', wards)
dhs_wards = get_regions(noisy_dhs_ward_poor.sort_values('rwi'), 0.1, 'ward', wards)
didl_wards = get_regions(didl_pls_ward.sort_values('didl_orig_rwi'), 0.1, 'ward', wards)


In [ ]:
#@title Plot regions at 10\% of pop targeted, original sample
# TODO: make partial a diff color?
import matplotlib.font_manager as fm
bold_font = fm.FontProperties(
    fname='drive/MyDrive/poverty_map_paper/Latin-Modern-Roman/lmroman10-bold.otf')

sns.set_style('white')
f, ax = plt.subplots(3, 2, figsize=(14, 24))
plt.subplots_adjust(wspace=0, hspace=-0.4)

for row in range(3):
    for col in range(2):
        country.plot(ax=ax[row][col], edgecolor='darkgray', color='none')
        for spine in ['top', 'bottom', 'left', 'right']:
            ax[row][col].spines[spine].set_visible(False)
        ax[row][col].xaxis.set_ticks([])
        ax[row][col].yaxis.set_ticks([])

    wards.plot(ax=ax[row][0], edgecolor='darkgray', color='none', linewidth=0.2)
    lgas.plot(ax=ax[row][1], edgecolor='darkgray', color='none', linewidth=0.2)
    
nlss_wards.plot(ax=ax[0][0], edgecolor='darkgray', color='black', linewidth=0.2)
didl_wards.plot(ax=ax[1][0], edgecolor='darkgray', color='#548b54', linewidth=0.2)
dhs_wards.plot(ax=ax[2][0], edgecolor='darkgray', color='lightblue', linewidth=0.2)

nlss_lgas.plot(ax=ax[0][1], edgecolor='darkgray', color='black', linewidth=0.2)
didl_lgas.plot(ax=ax[1][1], edgecolor='darkgray', color='#548b54', linewidth=0.2)
dhs_lgas.plot(ax=ax[2][1], edgecolor='darkgray', color='lightblue', linewidth=0.2)

ax[0][0].set_title('A. Wards Targeted', fontproperties=bold_font, size=18)
ax[0][1].set_title('B. LGAs Targeted', fontproperties=bold_font, size=18)

ax[0][0].set_ylabel('Optimal (NLSS)', fontproperties=bold_font, size=18)
ax[1][0].set_ylabel('ML-Based', fontproperties=bold_font, size=18)
ax[2][0].set_ylabel('Survey Benchmark', fontproperties=bold_font, size=18)
plt.savefig(FIG_PATH + 'targeted_units_10pct.png', dpi=300, bbox_inches='tight', pad_inches=0.2)


In [ ]:
#@title Make appendix table of overlap/wealth of targeted regions 

def frac_of_1_in_2(df1, df2, reg):
    overlap = df1[df1[reg].isin(df2[reg])]
    one_only = df1[~df1[reg].isin(df2[reg])]
    return round(len(overlap) / len(df1), 3)

    # rwi_overlap = (overlap['nlss_rwi'] * overlap['pop']).sum() / overlap['pop'].sum()
    # rwi_one_only = (one_only['nlss_rwi'] * one_only['pop']).sum() / one_only['pop'].sum()
    # print('Fraction of wards from one in two:', round(len(overlap) / len(df1), 3))
    # print('RWI of overlap:', round(rwi_overlap, 3))
    # print('RWI of one only:', round(rwi_one_only, 3))

def mean_wealth(df, rwi_df):
    wealth_df = df.merge(rwi_df)
    return round((wealth_df['nlss_rwi'] * wealth_df['pop']).sum() / wealth_df['pop'].sum(), 3)

lga_rwi = nlss_pls_lga[['lga', 'nlss_rwi']]
ward_rwi = nlss_pls_ward[['ward', 'nlss_rwi']]

print('LGAs')
print('Fraction of selected chosen under optimal targeting')
print(frac_of_1_in_2(nlss_lgas, nlss_lgas, 'lga'),
      frac_of_1_in_2(didl_lgas, nlss_lgas, 'lga'),
      frac_of_1_in_2(dhs_lgas, nlss_lgas, 'lga'))
print('Mean wealth, all selected')
print(mean_wealth(nlss_lgas, lga_rwi),
      mean_wealth(didl_lgas, lga_rwi),
      mean_wealth(dhs_lgas, lga_rwi))
print('Mean wealth, optimal selected')
print(mean_wealth(nlss_lgas[nlss_lgas['lga'].isin(nlss_lgas['lga'])], lga_rwi),
      mean_wealth(didl_lgas[didl_lgas['lga'].isin(nlss_lgas['lga'])], lga_rwi),
      mean_wealth(dhs_lgas[dhs_lgas['lga'].isin(nlss_lgas['lga'])], lga_rwi))
print('Mean wealth, non-optimal selected')
print('---',
      mean_wealth(didl_lgas[~didl_lgas['lga'].isin(nlss_lgas['lga'])], lga_rwi),
      mean_wealth(dhs_lgas[~dhs_lgas['lga'].isin(nlss_lgas['lga'])], lga_rwi))

print('\nWards')
print('Fraction of selected chosen under optimal targeting')
print(frac_of_1_in_2(nlss_wards, nlss_wards, 'ward'),
      frac_of_1_in_2(didl_wards, nlss_wards, 'ward'),
      frac_of_1_in_2(dhs_wards, nlss_wards, 'ward'))
print('Mean wealth, all selected')
print(mean_wealth(nlss_wards, ward_rwi),
      mean_wealth(didl_wards, ward_rwi),
      mean_wealth(dhs_wards, ward_rwi))
print('Mean wealth, optimal selected')
print(mean_wealth(nlss_wards[nlss_wards['ward'].isin(nlss_wards['ward'])], ward_rwi),
      mean_wealth(didl_wards[didl_wards['ward'].isin(nlss_wards['ward'])], ward_rwi),
      mean_wealth(dhs_wards[dhs_wards['ward'].isin(nlss_wards['ward'])], ward_rwi))
print('Mean wealth, non-optimal selected')
print('---',
      mean_wealth(didl_wards[~didl_wards['ward'].isin(nlss_wards['ward'])], ward_rwi),
      mean_wealth(dhs_wards[~dhs_wards['ward'].isin(nlss_wards['ward'])], ward_rwi))



In [ ]:
#@title Quantify overlap
def num_of_1_in_2(df1, df2, reg):
    overlap = df1[df1[reg].isin(df2[reg])]
    one_only = df1[~df1[reg].isin(df2[reg])]
    return len(overlap)

print(len(nlss_lgas), len(didl_lgas), len(dhs_lgas))
print(num_of_1_in_2(nlss_lgas, nlss_lgas, 'lga'), \
        num_of_1_in_2(didl_lgas, nlss_lgas, 'lga'), \
        num_of_1_in_2(dhs_lgas, nlss_lgas, 'lga'))
print(len(nlss_lgas) - num_of_1_in_2(nlss_lgas, nlss_lgas, 'lga'), \
        len(didl_lgas) - num_of_1_in_2(didl_lgas, nlss_lgas, 'lga'), \
        len(dhs_lgas) - num_of_1_in_2(dhs_lgas, nlss_lgas, 'lga'))

print(len(nlss_wards), len(didl_wards), len(dhs_wards))
print(num_of_1_in_2(nlss_wards, nlss_wards, 'ward'), \
        num_of_1_in_2(didl_wards, nlss_wards, 'ward'), \
        num_of_1_in_2(dhs_wards, nlss_wards, 'ward'))
print(len(nlss_wards) - num_of_1_in_2(nlss_wards, nlss_wards, 'ward'), \
        len(didl_wards) - num_of_1_in_2(didl_wards, nlss_wards, 'ward'), \
        len(dhs_wards) - num_of_1_in_2(dhs_wards, nlss_wards, 'ward'))

In [ ]:
#@title DIDL ward rank vs NLSS coverage data for R
didl_orig_ward['in_nlss'] = 0
didl_orig_ward.loc[didl_orig_ward['ward'].isin(nlss_ward['ward']), 'in_nlss'] = 1
didl_orig_ward = didl_orig_ward.sort_values('didl_orig_rwi')
didl_orig_ward['frac_in_nlss'] = didl_orig_ward['in_nlss'].expanding().mean()
didl_orig_ward['n_wards_targeted'] = didl_orig_ward['didl_orig_rwi'].rank()

didl_orig_ward.to_csv(OUT_DIR + 'ml_wards_in_nlss.csv', index=False)


In [ ]:
#@title Correlation between ML and NLSS wards furthest from DHS clusters
# warning - this takes ~6min

# nlss_ward_locs = wards[['ward', 'geometry']].merge(
#     nlss_ward[~nlss_ward['ward'].isin(dhs_ward['ward'])])
# nlss_ward_locs = nlss_ward_locs.to_crs(epsg=3310)
# ref = dhs_eval.to_crs(epsg=3310)

# nlss_ward_locs['nearest_dhs_cluster'] = nlss_ward_locs['geometry'].apply(
#     lambda x: ref['geometry'].apply(lambda y: x.distance(y)).min() / 1000
# )

# nlss_ward_locs.rename({'nearest_dhs_ward': 'nearest_dhs_cluster'}, axis=1)\
#     [['ward', 'nearest_dhs_cluster']].to_csv(
#     DATA_PATH + 'nlss/nearest_dhs_cluster.csv', index=False)

nlss_ward_locs = pd.read_csv(DATA_PATH + 'nlss/nearest_dhs_cluster.csv')\
                   .merge(didl_orig_ward[['ward', 'didl_orig_rwi']])\
                   .merge(nlss_ward[['ward', 'nlss_rwi']])\
                   .sort_values('nearest_dhs_cluster', ascending=False)
nlss_counts = nlss_hhs.groupby('ward')['hhid'].nunique().reset_index()
nlss_ward_locs = nlss_ward_locs.merge(nlss_counts)

print('All wards')
for i in [0, 1, 5, 10, 15, 20]:
    df = nlss_ward_locs[nlss_ward_locs['nearest_dhs_cluster'] >= i]
    rho, p, ci = get_r_ci(df, 'didl_orig_rwi', 'nlss_rwi')
    print('nearest DHS ward >=', i,
          '| n =', len(df),
          '| rho=%.3f (%.3f, %.3f), (p=%.2e)' % (rho, ci[0], ci[1], p))
    
# print('> 20 households')
# for i in [0, 1, 5, 10, 15, 20]:
#     df = nlss_ward_locs[(nlss_ward_locs['nearest_dhs_cluster'] >= i) &
#                     (nlss_ward_locs['hhid'] >= 20)]
#     rho, p, ci = get_r_ci(df, 'didl_orig_rwi', 'nlss_rwi')
#     print('nearest DHS ward >=', i,
#           '| n =', len(df),
#           '| rho=%.3f (%.3f, %.3f), (p=%.2e)' % (rho, ci[0], ci[1], p))

nlss_ward_locs = wards[['ward', 'geometry']]
nlss_ward_locs = nlss_ward_locs.to_crs(epsg=3310)
ref = dhs_eval.to_crs(epsg=3310)

nlss_ward_locs['nearest_dhs_cluster'] = nlss_ward_locs['geometry'].apply(
    lambda x: ref['geometry'].apply(lambda y: x.distance(y)).min() / 1000
)

nlss_ward_locs.rename({'nearest_dhs_ward': 'nearest_dhs_cluster'}, axis=1)\
    [['ward', 'nearest_dhs_cluster']].to_csv(
    DATA_PATH + 'nlss/nearest_dhs_cluster_all.csv', index=False)

nlss_ward_locs = nlss_ward_locs.merge(wards[['ward', 'state', 'pop']])
gt10 = nlss_ward_locs[nlss_ward_locs['nearest_dhs_cluster'] > 10]
print(1 - len(gt10) / len(nlss_ward_locs), 1 - gt10['pop'].sum() / nlss_ward_locs['pop'].sum())

gt10_nb = gt10[gt10['state'] != 'borno']
1 - len(gt10_nb) / len(nlss_ward_locs[nlss_ward_locs['state'] != 'borno']), \
1 - gt10_nb['pop'].sum() / nlss_ward_locs.loc[nlss_ward_locs['state'] != 'borno', 'pop'].sum()

get_r_ci(dhs_pls_ward_all, 'dhs_rwi', 'nlss_rwi')


In [ ]:
#@title ML <> NLSS far from DHS scatterplots
fig, ax = plt.subplots(1, 6, figsize=(24, 4))
cutoffs = [0, 1, 5, 10, 15, 20]

plt.title('abc')

for i in range(6):
    df = nlss_ward_locs[nlss_ward_locs['nearest_dhs_cluster'] >= cutoffs[i]]
    sns.scatterplot(x=df['didl_orig_rwi'], y=df['nlss_rwi'], ax=ax[i])
    ax[i].set_xlim(nlss_ward_locs['didl_orig_rwi'].min() - 0.1,
                   nlss_ward_locs['didl_orig_rwi'].max() + 0.1)
    ax[i].set_ylim(nlss_ward_locs['nlss_rwi'].min() - 0.1,
                   nlss_ward_locs['nlss_rwi'].max() + 0.1)
    ax[i].set_xlabel('ML-based wealth estimate')
    ax[i].set_ylabel('NLSS wealth estimate')
    if i == 0:
        ax[i].set_title('Anywhere outside ward')
    else:
        ax[i].set_title('%d km' % cutoffs[i])

fig.suptitle('Distance to nearest DHS cluster at least X km')




In [ ]:
#@title Correlation between DHS and NLSS wards furthest from DHS clusters

dhs_furthest_imputed = dhs_pls_ward_all[['ward', 'dhs_rwi', 'nlss_rwi']]\
    .merge(nlss_ward_locs[['ward', 'nearest_dhs_cluster']])

for i in [0, 1, 5, 10, 15, 20]:
    df = dhs_furthest_imputed[dhs_furthest_imputed['nearest_dhs_cluster'] >= i]
    rho, p, ci = get_r_ci(df, 'dhs_rwi', 'nlss_rwi')
    print('nearest DHS ward >=', i,
          '| n =', len(df),
          '| rho=%.3f (%.3f, %.3f), (p=%.2e)' % (rho, ci[0], ci[1], p))
    
# nearest DHS ward >= 0 | n = 1552 | rho=0.641 (0.611, 0.670), (p=1.97e-180)
# nearest DHS ward >= 1 | n = 1261 | rho=0.580 (0.543, 0.616), (p=1.88e-114)
# nearest DHS ward >= 5 | n = 643 | rho=0.411 (0.345, 0.474), (p=1.21e-27)
# nearest DHS ward >= 10 | n = 281 | rho=0.233 (0.119, 0.341), (p=7.95e-05)
# nearest DHS ward >= 15 | n = 118 | rho=0.318 (0.145, 0.471), (p=4.58e-04)
# nearest DHS ward >= 20 | n = 54 | rho=0.200 (-0.071, 0.444), (p=1.46e-01)

In [ ]:
#@title Get consumption histogram data for R
nlss_data.loc[nlss_data['totcons_adj'] < nlss_data['totcons_adj'].quantile(0.999),
              ['totcons_adj', 'popw']]\
    .rename({'totcons_adj': 'consumption', 'popw': 'weight'}, axis=1)\
    .to_csv(OUT_DIR + 'consumption_hist.csv', index=False)

df = nlss_data[nlss_data['totcons_adj'] < nlss_data['totcons_adj'].quantile(0.999)]
df = df[['totcons_adj', 'popw']]
ref = []
for i in range(len(df)):
    row = df.iloc[i]
    ref += [row['totcons_adj']] * int(round(row['popw'] / 1000))

ref_df = pd.DataFrame({'consumption': ref})
ref_df.to_csv(OUT_DIR + 'consumption_hist2.csv', index=False)

